## The Poisson Problem Script

In [ ]:
import matplotlib.pyplot as plt
import importlib
%matplotlib inline
if importlib.util.find_spec("matplotlib_inline") is not None:
    import matplotlib_inline
    matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
else:
    from IPython.display import set_matplotlib_formats
    set_matplotlib_formats('retina')

plt.ioff();

In [ ]:
import numpy as np
import torch
import json
import yaml
import os
import time
import shutil
import socket
import random
import chaospy
import pathlib
import fnmatch
import datetime
import resource
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorboardX
import psutil
from pyinstrument import Profiler
from torch import nn
from copy import deepcopy
from itertools import chain
from scipy.special import gamma
from os.path import exists, isdir
from collections import defaultdict
from collections import OrderedDict as odict
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
from bspinn.io_utils import DataWriter
from bspinn.io_utils import get_git_commit
from bspinn.io_utils import preproc_cfgdict
from bspinn.io_utils import hie2deep, deep2hie

from bspinn.tch_utils import isscalar
from bspinn.tch_utils import EMA
from bspinn.tch_utils import BatchRNG
from bspinn.tch_utils import bffnn
from bspinn.tch_utils import profmem

from bspinn.io_cfg import configs_dir
from bspinn.io_cfg import results_dir
from bspinn.io_cfg import storage_dir

## Theory

Consider the $d$-dimensional space $\mathbb{R}^{d}$, and the following charge:

$$\rho(x) = \delta^d(x).$$

For $d \neq 2$, the analytical solution to the system

$$\nabla \cdot \vec{E} = \rho$$

$$\nabla V = \vec{E}$$

can be defined as 

$$V_{\vec{x}} = \frac{\Gamma(d/2)}{2\cdot\pi^{d/2}\cdot (2-d)} \|\vec{x}\|^{2-d}, $$

$$\vec{E}_{\vec{x}} = \frac{\Gamma(d/2)}{2\cdot \pi^{d/2}\cdot \|\vec{x}\|^{d}} \vec{x}.$$

For $d=2$, $\vec{E}_{\vec{x}}$ is the same, but for $V_{\vec{x}}$ we have

$$V_{\vec{x}} = \frac{1}{2\pi} \ln(\|\vec{x}\|).$$

We want to solve this system using the divergence theorem:

$$\iint_{S_{d-1}(V)} \vec{E}\cdot \hat{n}\text{ d}S = \iiint_{V_d} \nabla.\vec{E}\text{ d}V.$$

Keep in mind that the $d-1$-dimensional surface of a $d$-dimensional shpere with radius $r$ is 
$$\iint_{S_{d-1}(V^{\text{d-Ball}}_{r})} 1\text{ d}S = \frac{2\cdot \pi^{d/2}}{\Gamma(d/2)}\cdot r^{d-1}.$$

### Dimensionality Scaling

We will assume that our domain of solution is a d-Ball centerred at zero with a radius of $r_b$.
$$C_1 := \int_{V_{r_b}^{d\text{-Ball}}} 1 d\vec{x} = \frac{2\pi^{d/2}}{d\cdot\Gamma(d/2)} r_b^d$$

#### The Expectation of the Anlytical Solution

$$E_v := \int_{V_r^{d\text{-Ball}}} V_{\vec{x}} d\vec{x} = \int \frac{\Gamma(d/2)}{2\cdot\pi^{d/2}\cdot (2-d)} \|\vec{x}\|^{2-d} d\vec{x}$$

$$ = C_1 \cdot \int \frac{\Gamma(d/2)}{2\cdot\pi^{d/2}\cdot (2-d)} \|\vec{x}\|^{2-d} \cdot \frac{1}{C_1} d\vec{x} $$

$$ = C_1 \cdot \frac{\Gamma(d/2)}{2\cdot\pi^{d/2}\cdot (2-d)} \int \|\vec{x}\|^{2-d} \cdot \frac{1}{C_1} d\vec{x} $$

$$ = \frac{r_b^d}{d\cdot(2-d)} \cdot \int \|\vec{x}\|^{2-d} \cdot \frac{1}{C_1} d\vec{x} $$

$$ = \frac{r_b^d}{d\cdot(2-d)} \cdot \mathbb{E}_{\vec{x}} [\|\vec{x}\|^{2-d}] $$

By defining the radius of $\vec{x}$ as $r=\|\vec{x}\|$, the distribution of $r$ is

$$Pr(\|\vec{x}\|<r) = (\frac{r}{r_b})^d$$

$$P(\|\vec{x}\|=r) = \frac{(d-1) \cdot r^d}{r_b^d}$$

Therefore, we have

$$E_v = \frac{r_b^d}{d\cdot(2-d)} \cdot \mathbb{E}_{\vec{x}} [r^{2-d}] $$

$$ = \frac{r_b^d}{d\cdot(2-d)} \cdot \int_{r=0}^{r_b} r^{2-d} \frac{(d-1) \cdot r^d}{r_b^d} dr$$

$$ = \frac{r_b^d}{d\cdot(2-d)} \cdot \frac{d}{r_b^d} \int_{r=0}^{r_b} r dr$$

$$ = \frac{r_b^d}{d\cdot(2-d)} \cdot \frac{d}{r_b^d} \int_{r=0}^{r_b} r dr$$

$$ = \frac{r_b^2}{2\cdot(2-d)}$$

#### The Expectation of the Volume Ratio

$$\mathbb{E}_{r\sim U[r_l, r_h]}[(\frac{r}{r_b})^d] = \frac{1}{r_h - r_l} \int_{r_l}^{r_h} (\frac{r}{r_b})^d dr$$

$$=\frac{1}{d+1} \cdot \frac{1}{r_b^d} \frac{r_h^{d+1} - r_l^{d+1}}{r_h - r_l}.$$

By setting $r_h=r_b$ and $r_l < r_h$, the above value closes in on $$\frac{1}{d+1}$$.

#### Spherical Coordinates

The spherical coordinate system in $d$ dimensions is relates $[x_1, \cdots, x_d]$ to $[r, \phi_1, \phi_2, \cdots, \phi_{d-1}]$ such that

$$x_1 = r \cos(\phi_1),$$
$$x_2 = r \sin(\phi_1)\cos(\phi_2),$$
$$x_2 = r \sin(\phi_1)\sin(\phi_2)\cos(\phi_3),$$
$$\ldots$$
$$x_{d-1} = r \sin(\phi_1)\sin(\phi_2)\cdots \cos(\phi_{d-1}),$$
$$x_d = r \sin(\phi_1)\sin(\phi_2)\cdots \sin(\phi_{d-1}).$$

We also have 
$$\phi_1, \cdots, \phi_{d-2} \in [0, \pi],$$ 
$$\phi_{d-1} \in [0, 2\pi).$$

The Jacobian determinent is therefore

$$\det([\frac{\partial x_i}{\partial(r, \phi_j)}]_{i,j}) = r^{d-1} \sin^{d-2}(\phi_1) \sin^{d-3}(\phi_2) \cdots \sin(\phi_{d-2}).$$

In other words, we have

$$ \text{d}^{d}(V) = r^{d-1} \sin^{d-2}(\phi_1) \sin^{d-3}(\phi_2) \cdots \sin(\phi_{d-2}) \text{d}r \text{d}\phi_1 \cdots \text{d} \phi_{d-1}.$$


#### Uniform Sphere Sampling
Define $F_n$ such that 

$$F_n(u) = \frac{1}{\int_{0}^{\pi} \sin^{n}(\phi) \text{d}{\phi}} \int_{0}^{\pi u} \sin^{n}(\phi) \text{d}{\phi}.$$

Note that $F_n(0) = 0$ and $F_n(1)=1$, and that $F_n$ is basically a CDF for a $\sin^{n}$-like PDF.

To sample points uniformly from the sphere of the $d$-dimensional unit ball, here is one process:

1. Sample $u = [u_1, u_2, \cdots, u_{d-1}]$ uniformly from $[0, 1]^{d-1}$.

2. Construct an inverse map for $F_n$ where $1 \leq n \leq {d-2}$. This can be done using a lookup table and `torch.searchsorted` for instance.

3. Compute the following:

$$\phi_1 = \pi \cdot F_{d-2}^{-1}(u_1),$$
$$\phi_2 = \pi \cdot F_{d-3}^{-1}(u_2),$$
$$\ldots$$
$$\phi_{d-2} = \pi \cdot F_{1}^{-1}(u_{d-2}),$$
$$\phi_{d-1} = 2 \pi \cdot u_{d-1}.$$

4. Translate $[r, \phi_1, \phi_2, \cdots, \phi_{d-1}]$ to the cartesian system $[x_1, \cdots, x_d]$.

Resources: 

1. https://en.wikipedia.org/wiki/N-sphere

2. http://corysimon.github.io/articles/uniformdistn-on-sphere/

### Defining the Problem and the Analytical Solution

In [ ]:
class DeltaProblem:
    def __init__(self, weights, locations, tch_device, tch_dtype):
        # weights          -> np.array -> shape=(n_bch, n_chrg)
        # locations.shape  -> np.array -> shape=(n_bch, n_chrg, d)
        self.weights = weights
        self.locations = locations
        self.n_bch, self.n_chrg = self.weights.shape
        self.d = self.locations.shape[-1]
        assert self.weights.shape == (self.n_bch, self.n_chrg,)
        assert self.locations.shape == (self.n_bch, self.n_chrg, self.d)
        self.weights_tch = torch.from_numpy(
            self.weights).to(tch_device, tch_dtype)
        self.locations_tch = torch.from_numpy(
            self.locations).to(tch_device, tch_dtype)
        self.shape = (self.n_bch,)
        self.ndim = 1
        self.tch_pi = torch.tensor(np.pi, device=tch_device, dtype=tch_dtype)

    def integrate_volumes(self, volumes):
        # volumes -> dictionary
        assert volumes['type'] == 'ball'
        centers = volumes['centers']
        radii = volumes['radii']
        n_v = radii.shape[-1]
        n_bch, n_chrg, d = self.n_bch, self.n_chrg, self.d
        assert radii.shape == (n_bch, n_v,)
        assert centers.shape == (n_bch, n_v, d)
        lib = torch if torch.is_tensor(centers) else np
        mu = self.locations_tch if torch.is_tensor(centers) else self.locations
        w = self.weights_tch if torch.is_tensor(centers) else self.weights

        c_diff_mu = centers.reshape(
            n_bch, n_v, 1, d) - mu.reshape(n_bch, 1, n_chrg, d)
        assert c_diff_mu.shape == (n_bch, n_v, n_chrg, d)
        distl2 = lib.sqrt(lib.square(c_diff_mu).sum(-1))
        assert distl2.shape == (n_bch, n_v, n_chrg)
        integ = ((distl2 < radii.reshape(n_bch, n_v, 1))
                 * w.reshape(n_bch, 1, n_chrg)).sum(-1)
        assert integ.shape == (n_bch, n_v)
        return integ

    def potential(self, x):
        lib = torch if torch.is_tensor(x) else np
        lib_pi = self.tch_pi if torch.is_tensor(x) else np.pi
        w = self.weights_tch if torch.is_tensor(x) else self.weights
        mu = self.locations_tch if torch.is_tensor(x) else self.locations
        n_bch, n_chrg, d = self.n_bch, self.n_chrg, self.d
        n_x = x.shape[-2]
        assert x.shape == (
            n_bch, n_x, d), f'x.shape={x.shape}, (n_bch, n_x, d)={(n_bch, n_x, d)}'
        x_diff_mu = x.reshape(n_bch, n_x, 1, d) - \
            mu.reshape(self.n_bch, 1, n_chrg, d)
        assert x_diff_mu.shape == (n_bch, n_x, n_chrg, d)
        x_dists = lib.sqrt(lib.square(x_diff_mu).sum(-1))
        assert x_dists.shape == (n_bch, n_x, n_chrg)
        if d != 2:
            poten1 = (x_dists**(2-d))
            assert poten1.shape == (n_bch, n_x, n_chrg)
            poten2 = (poten1 * w.reshape(n_bch, 1, n_chrg)).sum(-1)
            assert poten2.shape == (n_bch, n_x)
            cst = gamma(d/2) / (2*(lib_pi**(d/2)))
            cst = cst / (2-d)
            assert isscalar(cst)
            poten = cst * poten2
            assert poten.shape == (n_bch, n_x)
        else:
            poten1 = lib.log(x_dists)
            assert poten1.shape == (n_bch, n_x, n_chrg)
            poten2 = (poten1 * w.reshape(n_bch, 1, n_chrg)).sum(-1)
            assert poten2.shape == (n_bch, n_x)
            poten = poten2 / (2*lib_pi)
            assert poten.shape == (n_bch, n_x)
        return poten

    def field(self, x):
        lib = torch if torch.is_tensor(x) else np
        lib_pi = self.tch_pi if torch.is_tensor(x) else np.pi
        w = self.weights_tch if torch.is_tensor(x) else self.weights
        mu = self.locations_tch if torch.is_tensor(x) else self.locations
        n_bch, n_chrg, d = self.n_bch, self.n_chrg, self.d
        n_x = x.shape[-2]
        assert x.shape == (n_bch, n_x, d)
        x_diff_mu = x.reshape(n_bch, n_x, 1, d) - \
            mu.reshape(n_bch, 1, n_chrg, d)
        assert x_diff_mu.shape == (n_bch, n_x, n_chrg, d)
        x_dists = lib.sqrt(lib.square(x_diff_mu).sum(-1))
        assert x_dists.shape == (n_bch, n_x, n_chrg)
        poten1 = (x_dists**(-d))
        assert poten1.shape == (n_bch, n_x, n_chrg)
        poten2 = (poten1 * w.reshape(n_bch, 1, n_chrg)).sum(-1)
        assert poten2.shape == (n_bch, n_x)
        cst = gamma(d/2) / (2*(lib_pi**(d/2)))
        assert isscalar(cst)
        poten = cst * poten2
        assert poten.shape == (n_bch, n_x)
        field = poten.reshape(n_bch, n_x, 1) * x
        assert field.shape == (n_bch, n_x, d)
        return field



### Defining the Volume Sampler

In [ ]:
class BallSampler:
    def __init__(self, c_dstr, c_params, r_dstr, r_params, batch_rng):
        assert isinstance(c_params, dict)
        for name, param in c_params.items():
            msg_ = f'center param {name} is not np.array'
            assert isinstance(param, np.ndarray), msg_
        
        assert isinstance(r_params, dict)
        for name, param in r_params.items():
            msg_ = f'radius param {name} is not np.array'
            assert isinstance(param, np.ndarray), msg_

        self.batch_rng = batch_rng
        self.lib = batch_rng.lib
        
        ##############################################################
        ################# Center Sampling Parameters #################
        ##############################################################
        c_params_ = c_params.copy()
        self.c_dstr = c_dstr
        if c_dstr == 'uniform':
            c_low = c_params_.pop('low')
            c_high = c_params_.pop('high')
            
            n_bch, dim = c_low.shape
            
            self.c_low_np = c_low.reshape(n_bch, 1, dim)
            self.c_high_np = c_high.reshape(n_bch, 1, dim)
            self.c_size_np = (self.c_high_np - self.c_low_np)

            if self.lib == 'torch':
                self.c_low_tch = torch.from_numpy(self.c_low_np).to(
                    device=self.batch_rng.device, dtype=self.batch_rng.dtype)
                self.c_high_tch = torch.from_numpy(self.c_high_np).to(
                    device=self.batch_rng.device, dtype=self.batch_rng.dtype)
                self.c_size_tch = torch.from_numpy(self.c_size_np).to(
                    device=self.batch_rng.device, dtype=self.batch_rng.dtype)
            
            self.c_low = self.c_low_np if self.lib == 'numpy' else self.c_low_tch
            self.c_size = self.c_size_np if self.lib == 'numpy' else self.c_size_tch
        elif c_dstr == 'normal':
            c_loc = c_params_.pop('loc')
            c_scale = c_params_.pop('scale')
            
            n_bch, dim = c_loc.shape
            self.c_loc_np = c_loc.reshape(n_bch, 1, dim)
            self.c_scale_np = c_scale.reshape(n_bch, 1, 1)
            
            if self.lib == 'torch':
                self.c_loc_tch = torch.from_numpy(self.c_loc_np).to(
                    device=self.batch_rng.device, dtype=self.batch_rng.dtype)
                self.c_scale_tch = torch.from_numpy(self.c_scale_np).to(
                    device=self.batch_rng.device, dtype=self.batch_rng.dtype)
                
            self.c_loc = self.c_loc_np if self.lib == 'numpy' else self.c_loc_tch
            self.c_scale = self.c_scale_np if self.lib == 'numpy' else self.c_scale_tch
        elif c_dstr == 'ball':
            c_cntr = c_params_.pop('c')
            c_radi = c_params_.pop('r')
            
            n_bch, dim = c_cntr.shape
            self.c_cntr_np = c_cntr.reshape(n_bch, 1, dim)
            self.c_radi_np = c_radi.reshape(n_bch, 1, 1)
            
            if self.lib == 'torch':
                self.c_cntr_tch = torch.from_numpy(self.c_cntr_np).to(
                    device=self.batch_rng.device, dtype=self.batch_rng.dtype)
                self.c_radi_tch = torch.from_numpy(self.c_radi_np).to(
                    device=self.batch_rng.device, dtype=self.batch_rng.dtype)
                
            self.c_cntr = self.c_cntr_np if self.lib == 'numpy' else self.c_cntr_tch
            self.c_radi = self.c_radi_np if self.lib == 'numpy' else self.c_radi_tch
        else:
            raise ValueError(f'c_dstr="{c_dstr}" not implemented')
        
        msg_ = f'Some center parameters were left unused: {list(c_params_.keys())}'
        assert len(c_params_) == 0, msg_
            
        self.n_bch, self.d = n_bch, dim
        
        ##############################################################
        ################# Radius Sampling Parameters #################
        ##############################################################
        r_params_ = r_params.copy()
        r_low = r_params_.pop('low')
        r_high = r_params_.pop('high')
        
        if r_dstr == 'uniform':
            self.r_upow = 1.0
        elif r_dstr == 'unifdpow':
            self.r_upow = 1.0 / self.d
        else:
            raise ValueError(f'r_dstr={r_dstr} not implemented')

        r_low_rshp = r_low.reshape(self.n_bch, 1)
        r_high_rshp = r_high.reshape(self.n_bch, 1)
        assert (r_low >= 0.0).all()
        assert (r_high >= r_low).all()
        
        self.r_dstr = r_dstr
        self.r_low_np = np.power(r_low_rshp, 1.0/self.r_upow)
        self.r_high_np = np.power(r_high_rshp, 1.0/self.r_upow)
        self.r_size_np = (self.r_high_np - self.r_low_np)
        
        if self.lib == 'torch':
            self.r_low_tch = torch.from_numpy(self.r_low_np).to(
                device=self.batch_rng.device, dtype=self.batch_rng.dtype)
            self.r_high_tch = torch.from_numpy(self.r_high_np).to(
                device=self.batch_rng.device, dtype=self.batch_rng.dtype)
            self.r_size_tch = torch.from_numpy(self.r_size_np).to(
                device=self.batch_rng.device, dtype=self.batch_rng.dtype)
            
        self.r_low = self.r_low_np if self.lib == 'numpy' else self.r_low_tch
        self.r_size = self.r_size_np if self.lib == 'numpy' else self.r_size_tch
        
        msg_ = f'Some center parameters were left unused: {list(r_params_.keys())}'
        assert len(r_params_) == 0, msg_

    def __call__(self, n=1):
        radii = self.r_low + self.r_size * \
            self.batch_rng.uniform((self.n_bch, n))
        radii = radii ** self.r_upow
        
        if self.c_dstr == 'uniform':
            centers = self.batch_rng.uniform((self.n_bch, n, self.d))
            centers = centers * self.c_size + self.c_low
        elif self.c_dstr == 'normal':
            centers = self.batch_rng.normal((self.n_bch, n, self.d))
            centers = centers * self.c_scale + self.c_loc
        elif self.c_dstr == 'ball':
            rnd1 = self.batch_rng.normal((self.n_bch, n, self.d))
            rnd1 = rnd1 / ((rnd1**2).sum(-1, keepdims=True)**0.5)
            
            rnd2 = self.batch_rng.uniform((self.n_bch, n, 1))
            rnd2 = rnd2 ** (1./self.d)
            
            centers = self.c_radi * rnd2 * rnd1 + self.c_cntr
        else:
            raise ValueError(f'c_dstr="{self.c_dstr}" not implemented')
        
        d = dict()
        d['type'] = 'ball'
        d['centers'] = centers
        d['radii'] = radii
        return d



### Sruface Sampling

In [ ]:
class Cube2Sphere:
    def __init__(self, n_cdfint, dim, tch_device, tch_dtype):
        self.n_cdfint = n_cdfint
        self.dim = dim
        self.tch_device = tch_device
        self.tch_dtype = tch_dtype
        self.cdftab = self.get_cdftab(n_cdfint, dim)
        
    def tch_exlinspace(self, start, end, n):
        assert n >= 1
        a = torch.linspace(start, end, n+1,
                           device=self.tch_device,
                           dtype=self.tch_dtype)[:-1]
        b = a + 0.5 * (end - a[-1])
        return b
        
    def get_cdftab(self, n_cdfint, dim):
        r"""
        This function computes the necessary CDF functions $F_n$ for $1\leq n \leq dim-1$ such that 

            $$F_n(u) = \frac{1}{\int_{0}^{\pi} \sin^{n}(\phi) \text{d}{\phi}} \int_{0}^{\pi u} \sin^{n}(\phi) \text{d}{\phi}.$$

        Note that $F_n(0) = 0$ and $F_n(1)=1$, and that $F_n$ is basically a CDF for a $\sin^{n}$-like PDF.
        
        Parameters
        ----------
        n_cdfint: (int) The number of points for CDF integration and table lookup.

        dim: (int) The dimension of the space. The unit sphere should be a 
            `dim-1` dimensional manifold.
        """
        # Step 0: Defining the inverse CDF mapping
        thunif1d = self.tch_exlinspace(0.0, np.pi, n_cdfint)
        assert thunif1d.shape == (n_cdfint,)

        thunif = thunif1d.reshape(1, n_cdfint).expand(dim-1, n_cdfint)
        assert thunif.shape == (dim-1, n_cdfint)

        sinpow = torch.arange(dim-2, -1, -1, device=self.tch_device).reshape(dim-1, 1)
        assert sinpow.shape == (dim-1, 1)

        thsinpow = torch.sin(thunif) ** sinpow
        assert thsinpow.shape == (dim-1, n_cdfint)

        cdftab = thsinpow.cumsum(dim=-1)
        cdftab = cdftab / cdftab[:, -1:]
        assert cdftab.shape == (dim-1, n_cdfint)
        
        return cdftab
        
    def __call__(self, unifs):
        """
        Takes a set of uniform random values in the $[0, 1]^{dim-1}$ cube, and transforms it 
        to the points on the surface of the $d$-dimensional unit-ball.
        
        The transformation is designed in a way such that uniform inputs in the rectangle lead
        to uniform points on the surface of the unit-ball.
        
        Parameters
        ----------
        unifs: (torch.tensor) An input tensor with all values between 0 and 1. This input can 
            be batched. The shape of this tensor must end with `dim-1`.
            
            Example:
                `dim = 5`
                `unifs = torch.rand(100, 10, 25, 4)`
            
        Output
        ------
        x_cart: (torch.tensor) The output points on the unit sphere in the cartesian system.
            This will have the same batch dimensions as the input.
            
            Example:
                `dim = 5`
                `unifs = torch.rand(100, 10, 25, 4)`
                `assert x_cart.shape == (100, 10, 25, 5)`
        """
        dim, cdftab, n_cdfint = self.dim, self.cdftab, self.n_cdfint
        tch_dtype, tch_device = self.tch_dtype, self.tch_device
        assert unifs.shape[-1] == dim-1
        
        u_mbdims = unifs.shape[:-1]
        n_samp = np.prod(u_mbdims)
        
        # Step 2: Applying the inverse CDF mapping
        unifs_T = unifs.reshape(n_samp, dim-1).T.contiguous()
        assert unifs_T.shape == (dim-1, n_samp)

        cdfranks_T = torch.searchsorted(cdftab, unifs_T)
        assert cdfranks_T.shape == (dim-1, n_samp)

        cdfranks = cdfranks_T.T.reshape(n_samp, dim-1)
        assert cdfranks.shape == (n_samp, dim-1)

        cdfinvu = (cdfranks / n_cdfint).to(unifs.dtype)
        assert cdfinvu.shape == (n_samp, dim-1)

        # Step 3: Scaling the uniform values to the phi ranges
        #   Note: $\phi_{d-1}$ should be in the $[0, 2\pi]$ range, unlike 
        #         the $[0, \pi]$ range for the rest of the coordinates.
        phi_scale = torch.tensor([np.pi]*(dim-2) + [2*np.pi]).to(
            dtype=tch_dtype, device=tch_device).reshape(1, dim-1)
        assert phi_scale.shape == (1, dim-1)

        phi = cdfinvu * phi_scale
        assert phi.shape == (n_samp, dim-1)

        # Step 4: Translating to cartesian coordinates
        phi_cos_ = torch.cos(phi)
        assert phi_cos_.shape == (n_samp, dim-1)

        phi_cos = torch.cat([phi_cos_, torch.ones(n_samp, 1, 
            dtype=tch_dtype, device=tch_device)], dim=-1)
        assert phi_cos.shape == (n_samp, dim)

        phi_sin = torch.sin(phi)
        assert phi_sin.shape == (n_samp, dim-1)

        phi_sincumprod_ = phi_sin.cumprod(dim=-1)
        assert phi_sincumprod_.shape == (n_samp, dim-1)

        phi_sincumprod = torch.cat([torch.ones(n_samp, 1, dtype=tch_dtype, 
            device=tch_device), phi_sincumprod_], dim=-1)
        assert phi_sincumprod.shape == (n_samp, dim)

        x_cart_ = phi_sincumprod * phi_cos
        assert x_cart_.shape == (n_samp, dim)

        # Making sure the points lie on the unit sphere
        assert x_cart_.square().sum(dim=-1).allclose(torch.ones(1, dtype=tch_dtype, 
            device=tch_device))

        x_cart = x_cart_.reshape(*u_mbdims, dim)
        assert x_cart.shape == (*u_mbdims, dim)
        
        return x_cart
    


In [ ]:
class SphereSampler:
    def __init__(self, batch_rng):
        self.tch_dtype = batch_rng.dtype
        self.tch_device = batch_rng.device
        self.batch_rng = batch_rng

    def np_exlinspace(self, start, end, n):
        assert n >= 1
        a = np.linspace(start, end, n, endpoint=False)
        b = a + 0.5 * (end - a[-1])
        return b

    def tch_exlinspace(self, start, end, n):
        assert n >= 1
        a = torch.linspace(start, end, n+1,
                           device=self.tch_device,
                           dtype=self.tch_dtype)[:-1]
        b = a + 0.5 * (end - a[-1])
        return b

    def __call__(self, volumes, n, trnsfrm_params, samp_params, do_randrots=True, do_shflpts=True):
        # volumes -> dictionary
        assert volumes['type'] == 'ball'
        centers = volumes['centers']
        radii = volumes['radii']
        n_bch, n_v, d = centers.shape
        use_np = not torch.is_tensor(centers)
        assert centers.shape == (n_bch, n_v, d)
        assert radii.shape == (n_bch, n_v)
        assert not (use_np) or (self.batch_rng.lib == 'numpy')
        assert use_np or (self.batch_rng.device == centers.device)
        assert use_np or (self.batch_rng.dtype == centers.dtype)
        assert self.batch_rng.shape == (n_bch,)
        exlinspace = self.np_exlinspace if use_np else self.tch_exlinspace
        meshgrid = np.meshgrid if use_np else torch.meshgrid
        sin = np.sin if use_np else torch.sin
        cos = np.cos if use_np else torch.cos
        arccos = np.arccos if use_np else torch.arccos
        matmul = np.matmul if use_np else torch.matmul
        
        # Phase 0: Input arguments processing
        
        # Taking shallow copies
        trnsfrm_params, samp_params = dict(trnsfrm_params), dict(samp_params)
        trnsfrm_mthd = trnsfrm_params.pop('dstr')
        if trnsfrm_mthd == 'cube2sphr':
            cube2sphr = trnsfrm_params.pop('cube2sphr')
            rv_dim = d - 1
        elif trnsfrm_mthd == 'normscale':
            rv_dim = d
        else:
            raise RuntimeError('Not implemented')
        assert len(trnsfrm_params) == 0, f'unknown params: {trnsfrm_params}'
        
        samp_mthd = samp_params.pop('dstr')
        if samp_mthd == 'quad':
            quad_x = samp_params.pop('x')
            assert quad_x.shape == (n, rv_dim)
            quad_w = samp_params.pop('w')
            assert quad_w.shape == (n,)
            n_bch_, n_v_ = 1, 1
        elif samp_mthd == 'qmc':
            qmc_x = samp_params.pop('x')
            assert qmc_x.shape == (n, rv_dim)
            n_bch_, n_v_ = 1, 1
        elif samp_mthd == 'rng':
            n_bch_, n_v_ = n_bch, n_v
        elif samp_mthd == 'grid':
            n_bch_, n_v_ = 1, 1
        else:
            raise RuntimeError('Not implemented')
        assert len(samp_params) == 0, f'unknown params: {samp_params}'
        en_randrots = do_randrots and (samp_mthd in ('grid', 'quad', 'qmc'))
        en_shflpts = do_shflpts and (samp_mthd in ('quad', 'qmc', 'grid'))
        
        # Phase 1: Creating the `norms`, `unifs`, and `weights` variables.
        #          We either create or sample these variables, or process 
        #          the input arguments to create them.
        if (samp_mthd == 'grid') and (trnsfrm_mthd == 'normscale'):
            n_droot = int(np.round(n ** (1.0 / d)))
            assert n == (n_droot ** d), f'N={n} should have an integer {d} root (n_droot={n_droot})'
            u1d = exlinspace(0, 1, n_droot)
            assert u1d.shape == (n_droot,)
            n1d = torch.erfinv(2 * u1d - 1) * np.sqrt(2)
            norms = torch.cartesian_prod(*([n1d] * d)).reshape(1, 1, n, d)
            assert norms.shape == (n_bch_, n_v_, n, d)
            weights = torch.ones(1, 1, 1, dtype=self.tch_dtype, 
                device=self.tch_device).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        elif (samp_mthd == 'grid') and (trnsfrm_mthd == 'cube2sphr'):
            n_droot = int(np.round(n ** (1.0 / (d-1))))
            assert n == (n_droot ** (d-1)), f'N={n} should have an integer {d-1} root'
            u1d = exlinspace(0, 1, n_droot)
            assert u1d.shape == (n_droot,)
            unifs = torch.cartesian_prod(*([u1d] * (d-1))).reshape(1, 1, n, d-1)
            assert unifs.shape == (n_bch_, n_v_, n, d-1)
            weights = torch.ones(1, 1, 1, dtype=self.tch_dtype, 
                device=self.tch_device).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        elif (samp_mthd == 'rng') and (trnsfrm_mthd == 'normscale'):
            norms = self.batch_rng.normal((n_bch, n_v, n, d))
            assert norms.shape == (n_bch_, n_v_, n, d)
            weights = torch.ones(1, 1, 1, dtype=self.tch_dtype, 
                device=self.tch_device).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        elif (samp_mthd == 'rng') and (trnsfrm_mthd == 'cube2sphr'):
            unifs = self.batch_rng.uniform((n_bch, n_v, n, d-1))
            assert unifs.shape == (n_bch_, n_v_, n, d-1)
            weights = torch.ones(1, 1, 1, dtype=self.tch_dtype, 
                device=self.tch_device).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        elif (samp_mthd == 'quad') and (trnsfrm_mthd == 'normscale'):
            norms = quad_x.reshape(1, 1, n, d)
            assert norms.shape == (n_bch_, n_v_, n, d)
            weights = quad_w.reshape(1, 1, n).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        elif (samp_mthd  == 'quad') and (trnsfrm_mthd == 'cube2sphr'):            
            unifs = quad_x.reshape(1, 1, n, d-1)
            assert unifs.shape == (n_bch_, n_v_, n, d-1)
            weights = quad_w.reshape(1, 1, n).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        elif (samp_mthd == 'qmc') and (trnsfrm_mthd == 'normscale'):
            # norms = torch.erfinv(2 * qmc_x.reshape(1, 1, n, d) - 1) * np.sqrt(2)
            norms = qmc_x.reshape(1, 1, n, d)
            assert norms.shape == (n_bch_, n_v_, n, d)
            weights = torch.ones(1, 1, 1, dtype=self.tch_dtype, 
                device=self.tch_device).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        elif (samp_mthd == 'qmc') and (trnsfrm_mthd == 'cube2sphr'):
            unifs = qmc_x.reshape(1, 1, n, d-1)
            assert unifs.shape == (n_bch_, n_v_, n, d-1)
            weights = torch.ones(1, 1, 1, dtype=self.tch_dtype, 
                device=self.tch_device).expand(n_bch, n_v, n)
            assert weights.shape == (n_bch, n_v, n)
        else:
            raise RuntimeError('Not implemented yet!')

        # End of Phase 1. At this point, we should have the following 
        # satisfied under all conditions.
        assert weights.shape == (n_bch, n_v, n)
        if (trnsfrm_mthd == 'cube2sphr'):
            assert unifs.shape == (n_bch_, n_v_, n, d-1)
            assert (unifs >= 0.0).all()
            assert (unifs <= 1.0).all()
        else:
            assert norms.shape == (n_bch_, n_v_, n, d)
            assert (norms.square().sum(dim=-1) > 0).all()
            
        # Phase 2: Transforming `norms`/`unifs` to points on the unit-sphere.
        #          Inputs: `norms` and `unifs`.
        #          Outputs: `x_tilde_`
        if trnsfrm_mthd == 'normscale':
            norms_l2 = torch.sqrt(torch.square(norms).sum(dim=-1))
            x_tilde_ = norms / norms_l2.reshape(n_bch_, n_v_, n, 1)
            assert x_tilde_.shape == (n_bch_, n_v_, n, d)
        elif trnsfrm_mthd == 'cube2sphr':
            if cube2sphr is not None:
                assert cube2sphr is not None
                x_tilde_ = cube2sphr(unifs)
                assert x_tilde_.shape == (n_bch_, n_v_, n, d) 
            elif (cube2sphr is None) and (d in [2, 3]):
                if d == 2:
                    theta_2d = unifs * (2 * np.pi)
                    assert theta_2d.shape == (n_bch_, n_v_, n, 1)
                    x_tilde_list = [cos(theta_2d), sin(theta_2d)]
                elif d == 3:
                    u1_2d, u2_2d = unifs[..., :1], unifs[..., 1:]
                    assert u1_2d.shape == (n_bch_, n_v_, n, 1)
                    assert u2_2d.shape == (n_bch_, n_v_, n, 1)
                    phi_2d = arccos(1- 2 * u2_2d)
                    theta_2d = u1_2d * (2 * np.pi)
                    x_tilde_list = [sin(phi_2d) * cos(theta_2d),
                                    sin(phi_2d) * sin(theta_2d), 
                                    cos(phi_2d)]
                else:
                    raise RuntimeError('Not implemented yet!')
                if use_np:
                    x_tilde_ = np.concatenate(x_tilde_list, axis=-1)
                else:
                    x_tilde_ = torch.cat(x_tilde_list, dim=-1)
                assert x_tilde_.shape == (n_bch_, n_v_, n, d)
            else:
                raise RuntimeError('Not implemented yet!')
        else:
            raise RuntimeError('Not implemented yet!')
        
        # End of Phase 2. At this point, we should have the following
        # satisfied under all conditions
        assert x_tilde_.shape == (n_bch_, n_v_, n, d)
        
        # Phase 3: Final touches: random rotations, shuffling, constant calculations, etc.
        x_tilde = x_tilde_.expand(n_bch, n_v, n, d)
        assert x_tilde.shape == (n_bch, n_v, n, d)
        
        if en_shflpts:
            rngunifs = self.batch_rng.uniform((n_bch, n_v, n))
            assert rngunifs.shape == (n_bch, n_v, n)
            randperm3d = rngunifs.argsort(dim=-1)
            assert randperm3d.shape == (n_bch, n_v, n)
            randperm4d = randperm3d.reshape(n_bch, n_v, n, 1)
            assert randperm4d.shape == (n_bch, n_v, n, 1)
            
            x_tilde_shfld = torch.take_along_dim(x_tilde, randperm4d, dim=-2)
            assert x_tilde_shfld.shape == (n_bch, n_v, n, d)
            weights_shfld = torch.take_along_dim(weights, randperm3d, dim=-1)
            assert weights_shfld.shape == (n_bch, n_v, n)
        else:
            x_tilde_shfld = x_tilde
            assert x_tilde_shfld.shape == (n_bch, n_v, n, d)
            weights_shfld = weights
            assert weights_shfld.shape == (n_bch, n_v, n)

        if en_randrots:
            rot_mats = self.batch_rng.so_n((n_bch, n_v, d, d))
            assert rot_mats.shape == (n_bch, n_v, d, d)
            
        if en_randrots:
            x_tilde_rot = matmul(x_tilde_shfld, rot_mats)
        else:
            x_tilde_rot = x_tilde_shfld
        assert x_tilde_rot.shape == (n_bch, n_v, n, d)

        points = x_tilde_rot * \
            radii.reshape(n_bch, n_v, 1, 1) + centers.reshape(n_bch, n_v, 1, d)
        assert points.shape == (n_bch, n_v, n, d)

        if use_np:
            x_tilde_bc = np.broadcast_to(x_tilde_shfld, (n_bch, n_v, n, d))
        else:
            x_tilde_bc = x_tilde_shfld.expand(n_bch, n_v, n, d)

        if en_randrots:
            rot_x_tilde = matmul(x_tilde_bc, rot_mats)
        else:
            rot_x_tilde = x_tilde_bc
        assert rot_x_tilde.shape == (n_bch, n_v, n, d)

        cst = (2*(np.pi**(d/2))) / gamma(d/2)
        csts = cst * (radii**(d-1))
        assert csts.shape == (n_bch, n_v)

        ret_dict = dict(points=points, normals=rot_x_tilde, weights=weights_shfld, areas=csts)
        return ret_dict



### Visualization

In [ ]:
def get_nn_sol(model, x, n_eval=None, get_field=True, 
    out_lib='numpy'):
    """
    Gets a model and evaluates it minibatch-wise on the tensor x. 
    The minibatch size is capped at n_eval. The output will have the 
    predicted potentials and the vector fields at them.

    Parameters
    ----------
    model: (nn.module) the batched neural network.

    x: (torch.tensor) the evaluation points. This array should be 
        >2-dimensional and have a shape of `(..., x_rows, x_cols)`.

    n_eval: (int or None) the maximum mini-batch size. If None is 
        given, `x_rows` will be used as `n_eval`.
        
    out_lib: (str) determines the output tensor type. Should be either 
        'numpy' or 'torch'.
    
    Output Dictionary
    ----------
    v: (np.array or torch.tensor) the evaluated potentials 
        with a shape of `(*model.shape, x_rows)` where
        model.shape is the batch dimensions of the model. 

    e: (np.array or torch.tensor) the evaluated vector fields 
        with a shape of `(*model.shape, x_rows, x_cols)` where
        model.shape is the batch dimensions of the model.
    """
    x_rows, x_cols = tuple(x.shape)[-2:]
    x_bd_ = tuple(x.shape)[:-2]
    x_bd = (1,) if len(x_bd_) == 0 else x_bd_
    msg_ = f'Cannot have {x.shape} fed to {model.shape}'
    assert len(x_bd) <= model.ndim, msg_
    if len(x_bd) < model.ndim:
        x_bd = tuple([1] * (model.ndim-len(x_b)) + list(x_bd))
    assert all((a == b) or (a == 1) or (b == 1) 
               for a, b in zip(x_bd, model.shape)), msg_
    n_eval = x_rows if n_eval is None else n_eval
    if out_lib == 'numpy':
        to_lib = lambda a: a.detach().cpu().numpy()
        lib_cat = lambda al: np.concatenate(al, axis=1)
        lpf = '_np'
    elif out_lib == 'torch':
        to_lib = lambda a: a
        lib_cat = lambda al: torch.cat(al, dim=1)
        lpf = ''
    else:
        raise ValueError(f'outlib={outlib} not defined.')

    n_batches = int(np.ceil(x_rows / n_eval))
    v_pred_list = []
    e_pred_list = []
    for i in range(n_batches):
        x_i = x[..., (i*n_eval):((i+1)*n_eval), :]
        xi_rows = x_i.shape[-2]
        x_ii = x_i.reshape(*x_bd, xi_rows, x_cols)
        x_iii = x_ii.expand(*model.shape, xi_rows, x_cols)
        x_iiii = nn.Parameter(x_iii)
        v_pred_i = model(x_iiii).squeeze(-1)
        v_pred_ii = to_lib(v_pred_i.detach())
        v_pred_list.append(v_pred_ii)
        if get_field:
            e_pred_i, = torch.autograd.grad(v_pred_i.sum(), [x_iiii],
                grad_outputs=None, retain_graph=False, create_graph=False,
                only_inputs=True, allow_unused=False)
            e_pred_ii = to_lib(e_pred_i.squeeze(-1).detach())
            e_pred_list.append(e_pred_ii)

    v_pred = lib_cat(v_pred_list)
    if get_field:
        e_pred = lib_cat(e_pred_list)
    else:
        e_pred = None

    outdict = {f'v{lpf}': v_pred, f'e{lpf}': e_pred}
    return outdict


def get_prob_sol(problem, x, n_eval=None, get_field=True, 
    out_lib='numpy'):
    """
    Gets a problem and evaluates the analytical solution to its 
    potentials and vector fields minibatch-wise on the tensor x. 
    The minibatch size is capped at n_eval. The output will have the 
    predicted potentials and the vector fields at them.

    Parameters
    ----------
    problem: (object) the problem with both the `potential` and 
        `field` methods for analytical solution evaluation.

    x: (torch.tensor) the evaluation points. This array should be 
        >2-dimensional and have a shape of `(..., x_rows, x_cols)`.

    n_eval: (int or None) the maximum mini-batch size. If None is 
        given, `x_rows` will be used as `n_eval`.

    Output Dictionary
    ----------
    v_np: (np.array) the evaluated potentials with a shape of
        `(..., x_rows)`. 

    e_np: (np.array) the evaluated vector fields with a shape of
        `(..., x_rows, x_cols)`.
    """

    assert hasattr(problem, 'potential')
    assert callable(problem.potential)
    assert hasattr(problem, 'field')
    assert callable(problem.field)

    x_rows, x_cols = tuple(x.shape)[-2:]
    x_bd_ = tuple(x.shape)[:-2]
    x_bd = (1,) if len(x_bd_) == 0 else x_bd_
    msg_ = f'Cannot have {x.shape} fed to {problem.shape}'
    assert len(x_bd) <= problem.ndim, msg_
    if len(x_bd) < problem.ndim:
        x_bd = tuple([1] * (problem.ndim-len(x_b)) + list(x_bd))
    assert all((a == b) or (a == 1) or (b == 1) 
               for a, b in zip(x_bd, problem.shape)), msg_
    n_eval = x_rows if n_eval is None else n_eval
    if out_lib == 'numpy':
        to_lib = lambda a: a.detach().cpu().numpy()
        lib_cat = lambda al: np.concatenate(al, axis=1)
        lpf = '_np'
    elif out_lib == 'torch':
        to_lib = lambda a: a
        lib_cat = lambda al: torch.cat(al, dim=1)
        lpf = ''
    else:
        raise ValueError(f'outlib={outlib} not defined.')

    n_batches = int(np.ceil(x_rows / n_eval))
    v_list = []
    e_list = []
    for i in range(n_batches):
        x_i = x[..., (i*n_eval):((i+1)*n_eval), :]
        xi_rows = x_i.shape[-2]
        x_ii = x_i.reshape(*x_bd, xi_rows, x_cols)
        x_iii = x_ii.expand(*problem.shape, xi_rows, x_cols)
        v_i = problem.potential(x_iii)
        v_list.append(to_lib(v_i))
        if get_field:
            e_i = problem.field(x_iii)
            e_list.append(to_lib(e_i))

    v = lib_cat(v_list)
    if get_field:
        e = lib_cat(e_list)
    else:
        e = None
    outdict = {f'v{lpf}': v, f'e{lpf}': e}
    return outdict


def make_grid(x_low, x_high, dim, n_gpd, lib):
    """
    Creates a grid of points using the mesgrid functions
    
    Parameters
    ----------
    x_low: (list) a list of length `dim` with floats 
        representing the lower limits of the grid.
    
    x_high: (list) a list of length `dim` with floats 
        representing the higher limits of the grid.
    
    dim: (int) the dimension of the grid space.
    
    n_gpd: (int) the number of points in each 
        grid dimension. This yields a total of 
        `n_gpd**dim` points in the total grid.
        
    lib: (str) either 'torch' or 'numpy'. This determines 
        the type of `x` output.
        
    Outputs
    -------
    x: (torch.tensor or np.array) a 2-d tensor or array 
        with the shape of `(n_gpd**dim, dim)`. 
    
    xi_msh_np: (list of np.array) a list of length `dim` 
        with meshgrid tensors each with a shape of 
        `[n_gpd] * dim`.
    """
    
    assert dim == 2, 'not implemented yet'
    assert len(x_low) == dim
    assert len(x_high) == dim
    assert lib in ('torch', 'numpy')
    library = torch if lib == 'torch' else np
    tnper = lambda a: a.cpu().detach().numpy()
    nper = tnper if lib == 'torch' else lambda a: a
    
    x1_low, x2_low = x_low
    x1_high, x2_high = x_high
    n_g_plt = n_gpd ** dim

    x1_1d = library.linspace(x1_low, x1_high, n_gpd)
    assert x1_1d.shape == (n_gpd,)

    x2_1d = library.linspace(x2_low, x2_high, n_gpd)
    assert x2_1d.shape == (n_gpd,)

    x1_msh, x2_msh = library.meshgrid(x1_1d, x2_1d)
    assert x1_msh.shape == (n_gpd, n_gpd)
    assert x2_msh.shape == (n_gpd, n_gpd)

    x1 = x1_msh.reshape(n_g_plt, 1)
    assert x1.shape == (n_g_plt, 1)

    x2 = x2_msh.reshape(n_g_plt, 1)
    assert x2.shape == (n_g_plt, 1)

    x1_1d_c = x1_1d.reshape(n_gpd, 1)
    assert x1_1d_c.shape == (n_gpd, 1)

    x2_1d_c = x2_1d.reshape(n_gpd, 1)
    assert x2_1d_c.shape == (n_gpd, 1)

    x1_msh_np = nper(x1_msh)
    assert x1_msh_np.shape == (n_gpd, n_gpd)

    x2_msh_np = nper(x2_msh)
    assert x1_msh_np.shape == (n_gpd, n_gpd)

    x = torch.cat([x1, x2], dim=1)
    assert x.shape == (n_g_plt, dim)

    x_np = nper(x)
    assert x_np.shape == (n_g_plt, dim)
    
    xi_msh_np = [x1_msh_np, x2_msh_np]
    outdict = dict(x=x, xi_msh_np=xi_msh_np)

    return outdict


def plot_sol(x1_msh_np, x2_msh_np, sol_dict, fig=None, ax=None, cax=None):
    n_gpd, dim = x1_msh_np.shape[0], x1_msh_np.ndim
    assert dim == 2, f'dim={dim}, x1_msh_np.shape={x1_msh_np.shape}'
    assert x1_msh_np.shape == (n_gpd, n_gpd)
    assert x2_msh_np.shape == (n_gpd, n_gpd)
    n_g = (n_gpd ** dim)
   
    if fig is None:
        assert ax is None
        assert cax is None
        fig, ax = plt.subplots(1, 1, figsize=(3.0, 2.5), dpi=72)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.05)
    else:
        assert ax is not None
   
    e_percentile_cap = 90
    if 'v_np' in sol_dict:
        v_np = sol_dict['v_np']
    else:
        v_np = sol_dict['v'].detach().cpu().numpy()
    assert v_np.shape[-1] == n_g
    
    v_msh_np = v_np.reshape(-1, n_gpd, n_gpd).mean(axis=0)
    im = ax.pcolormesh(x1_msh_np, x2_msh_np, v_msh_np,
                        shading='auto', cmap='RdBu')
    if cax is not None:
        fig.colorbar(im, cax=cax)

    if 'e_np' in sol_dict:
        e_msh_np = sol_dict['e_np']
    else:
        e_msh_np = sol_dict['e']
        if e_msh_np is not None:
            e_msh_np = e_msh_np.detach().cpu().numpy()
    
    if e_msh_np is not None:
        assert e_msh_np.shape[-2:] == (n_g, dim)
        e_msh_np = e_msh_np.reshape(-1, n_gpd,
            n_gpd, dim).mean(axis=0)
        if e_percentile_cap is not None:
            e_size = np.sqrt((e_msh_np**2).sum(axis=-1))
            e_size_cap = np.percentile(a=e_size, 
                q=e_percentile_cap, axis=None)
            cap_coef = np.ones_like(e_size)
            cap_coef[e_size > e_size_cap] = e_size_cap / \
                e_size[e_size > e_size_cap]
            e_msh_capped = e_msh_np * \
                cap_coef.reshape(*e_msh_np.shape[:-1], 1)
        else:
            e_msh_capped = e_msh_np

        ax.quiver(x1_msh_np, x2_msh_np,
            e_msh_capped[:, :, 0], e_msh_capped[:, :, 1])
    return fig, ax, cax


def get_perfdict(e_pnts, e_mdlsol, e_prbsol):
    """
    Computes the biased, bias-corrected, and slope-corrected error 
    metrics for the solutions of a Poisson problem.
    
    This function computes three types of MSE and MAE statistics:
        
        1. Plain: just take the model and ground truth solution
            and subtract them to get the errors. No bias- or slope-correction 
            is applied to offset those degrees of freedom.
            
            shorthand: 'pln'
            
        2. Bias-corrected: subtracts the average value from both the model 
            and ground truth solutions, and then computes the errors.
            
            shorthand: 'bc'
            
        3. Slope-corrected: Since any linear function can be added to the
            Poisson solutions without violating the poisson equation, this
            function fits an ordinary least squares to both the model and
            ground truth solutions, and then subtracts it from them. This
            way, even the arbitrary-slope issue can be addressed.
            
            shorthand: 'slc'
            
    Parameters
    ----------
    e_pnts: (torch.tensor) The input points to the model and the ground truth.
        This should have a shape of (n_seeds, n_evlpnts, dim).
        
    e_mdlsol: (torch.tensor) The model solution with a
        (n_seeds, n_evlpnts) shape.
    
    e_prbsol: (torch.tensor) The ground truth solution with a
        (n_seeds, n_evlpnts) shape.
        
    Output
    ------
    outdict: (dict) A mapping between the error keys and their numpy arrays.
        The error keys are the cartesian product of ('pln', 'bc', 'slc') 
        and ('mse', 'mae').
    """
    n_seeds, n_evlpnts, dim = e_pnts.shape
    assert e_mdlsol.shape == (n_seeds, n_evlpnts)
    assert e_prbsol.shape == (n_seeds, n_evlpnts)
    
    with torch.no_grad():
        # The plain non-processed error matrix
        err_pln = e_mdlsol - e_prbsol
        assert err_pln.shape == (n_seeds, n_evlpnts)
        
        # The bias-corrected error matrix
        e_mdlsol2 = e_mdlsol - e_mdlsol.mean(dim=-1, keepdims=True)
        assert e_mdlsol2.shape == (n_seeds, n_evlpnts)
        e_prbsol2 = e_prbsol - e_prbsol.mean(dim=-1, keepdims=True)
        assert e_prbsol2.shape == (n_seeds, n_evlpnts)
        err_bc = e_mdlsol2 - e_prbsol2
        assert err_bc.shape == (n_seeds, n_evlpnts)
        
        # The bias-corrected and normalized error matrix
        e_mdlsol3 = e_mdlsol2 / e_mdlsol2.square().mean(dim=-1, keepdims=True).sqrt()
        assert e_mdlsol3.shape == (n_seeds, n_evlpnts)
        e_prbsol3 = e_prbsol2 / e_prbsol2.square().mean(dim=-1, keepdims=True).sqrt()
        assert e_prbsol3.shape == (n_seeds, n_evlpnts)
        err_bcn = e_mdlsol3 - e_prbsol3
        assert err_bcn.shape == (n_seeds, n_evlpnts)
        
        # The slope-corrected error matrix
        e_pntstrans = e_pnts.transpose(-1, -2)
        assert e_pntstrans.shape == (n_seeds, dim, n_evlpnts)
        e_pntsig = e_pntstrans.matmul(e_pnts)
        assert e_pntsig.shape == (n_seeds, dim, dim)
        e_pntsiginv = torch.pinverse(e_pntsig)
        assert e_pntsiginv.shape == (n_seeds, dim, dim)
        e_pntpinv = e_pntsiginv.matmul(e_pntstrans)
        assert e_pntpinv.shape == (n_seeds, dim, n_evlpnts)
        
        # e_pntpinv = torch.pinverse(e_pnts)
        # assert e_pntpinv.shape == (n_seeds, dim, n_evlpnts)

        e_mdlbeta = e_pntpinv.matmul(e_mdlsol2.unsqueeze(-1))
        assert e_mdlbeta.shape == (n_seeds, dim, 1)
        e_mdlslpcrc = e_pnts.matmul(e_mdlbeta)
        assert e_mdlslpcrc.shape == (n_seeds, n_evlpnts, 1)
        e_mdlsol4 = e_mdlsol2 - e_mdlslpcrc.squeeze(-1)
        assert e_mdlsol4.shape == (n_seeds, n_evlpnts)
        
        e_prbbeta = e_pntpinv.matmul(e_prbsol2.unsqueeze(-1))
        assert e_prbbeta.shape == (n_seeds, dim, 1)
        e_prbslpcrc = e_pnts.matmul(e_prbbeta)
        assert e_prbslpcrc.shape == (n_seeds, n_evlpnts, 1)
        e_prbsol4 = e_prbsol2 - e_prbslpcrc.squeeze(-1)
        assert e_prbsol4.shape == (n_seeds, n_evlpnts)
        
        err_slc = e_mdlsol4 - e_prbsol4
        assert err_slc.shape == (n_seeds, n_evlpnts)
        
        # The normalized slope-corrected error matrix
        e_mdlsol5 = e_mdlsol4 - e_mdlsol4.mean(dim=-1, keepdims=True)
        assert e_mdlsol5.shape == (n_seeds, n_evlpnts)
        
        e_mdlsol6 = e_mdlsol5 / e_mdlsol5.square().mean(dim=-1, keepdims=True).sqrt()
        assert e_mdlsol6.shape == (n_seeds, n_evlpnts)
        
        e_prbsol5 = e_prbsol4 - e_prbsol4.mean(dim=-1, keepdims=True)
        assert e_prbsol5.shape == (n_seeds, n_evlpnts)
        
        e_prbsol6 = e_prbsol5 / e_prbsol5.square().mean(dim=-1, keepdims=True).sqrt()
        assert e_prbsol6.shape == (n_seeds, n_evlpnts)
        
        err_scn = e_mdlsol6 - e_prbsol6
        assert err_scn.shape == (n_seeds, n_evlpnts)
        
        # Computing the mse and mae values
        e_plnmse = err_pln.square().mean(dim=-1)
        assert e_plnmse.shape == (n_seeds,)
        e_plnmae = err_pln.abs().mean(dim=-1)
        assert e_plnmse.shape == (n_seeds,)
        
        e_bcmse = err_bc.square().mean(dim=-1)
        assert e_bcmse.shape == (n_seeds,)
        e_bcmae = err_bc.abs().mean(dim=-1)
        assert e_bcmse.shape == (n_seeds,)
        
        e_bcnmse = err_bcn.square().mean(dim=-1)
        assert e_bcnmse.shape == (n_seeds,)
        e_bcnmae = err_bcn.abs().mean(dim=-1)
        assert e_bcnmse.shape == (n_seeds,)
        
        e_slcmse = err_slc.square().mean(dim=-1)
        assert e_slcmse.shape == (n_seeds,)
        e_slcmae = err_slc.abs().mean(dim=-1)
        assert e_slcmse.shape == (n_seeds,)
        
        e_scnmse = err_scn.square().mean(dim=-1)
        assert e_scnmse.shape == (n_seeds,)
        e_scnmae = err_scn.abs().mean(dim=-1)
        assert e_scnmse.shape == (n_seeds,)
    
        outdict = {'pln/mse': e_plnmse.detach().cpu().numpy(),
                   'pln/mae': e_plnmae.detach().cpu().numpy(),
                   'bc/mse': e_bcmse.detach().cpu().numpy(),
                   'bc/mae': e_bcmae.detach().cpu().numpy(),
                   'bcn/mse': e_bcnmse.detach().cpu().numpy(),
                   'bcn/mae': e_bcnmae.detach().cpu().numpy(),
                   'slc/mse': e_slcmse.detach().cpu().numpy(),
                   'slc/mae': e_slcmae.detach().cpu().numpy(),
                   'scn/mse': e_scnmse.detach().cpu().numpy(),
                   'scn/mae': e_scnmae.detach().cpu().numpy()}
    
    return outdict

## Optional Visualization Tests

### Visualizing the True Potential and Fields

In [ ]:
ex_device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
ex_tchdevice = torch.device(ex_device)
ex_tchdtype = torch.double
prob2d_ex1 = DeltaProblem(weights=np.array([[1.0, 1.0, 1.0]]),
                          locations=np.array([[[0.0,  0.0],
                                               [-0.5, -0.5],
                                               [0.5,  0.5]]]),
                          tch_device=ex_tchdevice,
                          tch_dtype=ex_tchdtype)

fig, ax = plt.subplots(1, 1, figsize=(3.0, 2.5), dpi=72)
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)

ex_gdict = make_grid(x_low=[-1, -1], x_high=[1., 1.], 
    dim=2 , n_gpd=50, lib='torch')
x = ex_gdict['x'].to(ex_tchdevice, ex_tchdtype)
x1_msh_np, x2_msh_np = ex_gdict['xi_msh_np']

ex_sol = get_prob_sol(prob2d_ex1, x, n_eval=200, get_field=True)
fig, ax, cax = plot_sol(x1_msh_np, x2_msh_np, ex_sol, 
    fig=fig, ax=ax, cax=cax)
fig

### Visualizing the Sampler and Integrator

In [ ]:
n_bch = 5
rng = BatchRNG(shape=(n_bch,), lib='torch', device=ex_tchdevice, dtype=ex_tchdtype,
               unif_cache_cols=100, norm_cache_cols=500)
rng.seed(np.broadcast_to(12345+np.arange(n_bch), rng.shape))

prob2d_ex2 = DeltaProblem(weights=np.broadcast_to([1.0, 1.0, 1.0], (n_bch, 3)).copy(),
                          locations=np.broadcast_to([[0.0,  0.0],
                                                     [-0.5, -0.5],
                                                     [0.5,  0.5]], (n_bch, 3, 2)).copy(),
                          tch_device=ex_tchdevice, tch_dtype=ex_tchdtype)

volsampler_2d = BallSampler(c_dstr='uniform', c_params=dict(
                            low=np.broadcast_to([-1.0, -1.0], (n_bch, 2)).copy(),
                            high=np.broadcast_to([1.0,  1.0], (n_bch, 2)).copy()),
                            r_dstr='uniform', r_params=dict(
                            low=np.broadcast_to([0.1], (n_bch,)).copy(),
                            high=np.broadcast_to([1.5], (n_bch,)).copy()),
                            batch_rng=rng)

vols = volsampler_2d(n=10)
integs = prob2d_ex2.integrate_volumes(vols)
for key, val in vols.items():
    if torch.is_tensor(val):
        vols[key] = val.detach().cpu().numpy()

In [ ]:
fig = plt.figure(dpi=36)
ax = plt.gca()

i = 0
max_integ = prob2d_ex2.weights[i][prob2d_ex2.weights[i] > 0].sum()
min_integ = prob2d_ex2.weights[i][prob2d_ex2.weights[i] < 0].sum()
cmap = mpl.cm.get_cmap('RdBu')
cnorm = mpl.colors.Normalize(vmin=min_integ, vmax=max_integ)

ax.scatter(prob2d_ex2.locations[i, :, 0],
           prob2d_ex2.locations[i, :, 1], marker='*', color='black', s=150)
for center, radius, integ in zip(vols['centers'][i], vols['radii'][i], integs[i]):
    circle = plt.Circle(center, radius, fill=False,
                        color=cmap(1.0-cnorm(integ.item())))
    ax.add_patch(circle)
ax.set_aspect('equal', adjustable='box')
fig

In [ ]:
n_bch = 5
rng = BatchRNG(shape=(n_bch,), lib='torch', device=ex_tchdevice, dtype=ex_tchdtype,
               unif_cache_cols=100, norm_cache_cols=500)
rng.seed(np.broadcast_to(12345+np.arange(n_bch), rng.shape))

prob2d_ex3 = DeltaProblem(weights=np.broadcast_to([1.0, 1.0, 1.0], (n_bch, 3)).copy(),
                          locations=np.broadcast_to([[0.0,  0.0],
                                                     [-0.5, -0.5],
                                                     [0.5,  0.5]], (n_bch, 3, 2)).copy(),
                          tch_device=ex_tchdevice,
                          tch_dtype=ex_tchdtype)


volsampler_2d = BallSampler(c_dstr='uniform', c_params=dict(
                            low=np.broadcast_to([-1.0, -1.0], (n_bch, 2)).copy(),
                            high=np.broadcast_to([1.0,  1.0], (n_bch, 2)).copy()),
                            r_dstr='uniform', r_params=dict(
                            low=np.broadcast_to([0.1], (n_bch,)).copy(),
                            high=np.broadcast_to([1.5], (n_bch,)).copy()),
                            batch_rng=rng)

sphsampler_2d = SphereSampler(batch_rng=rng)

vols = volsampler_2d(n=10)
sphsamps2d = sphsampler_2d(vols, 100, 
    trnsfrm_params=dict(dstr='cube2sphr', cube2sphr=None), 
    samp_params=dict(dstr='grid'), do_randrots=True, do_shflpts=False)
points = sphsamps2d['points']
surfacenorms = sphsamps2d['normals']
if torch.is_tensor(points):
    points = points.detach().cpu().numpy()
if torch.is_tensor(surfacenorms):
    surfacenorms = surfacenorms.detach().cpu().numpy()
points.shape, surfacenorms.shape

In [ ]:
fig = plt.figure(dpi=36)
ax = plt.gca()

i = 0

max_integ = prob2d_ex3.weights[i][prob2d_ex3.weights[i] > 0].sum()
min_integ = prob2d_ex3.weights[i][prob2d_ex3.weights[i] < 0].sum()
cmap = mpl.cm.get_cmap('RdBu')
cnorm = mpl.colors.Normalize(vmin=min_integ, vmax=max_integ)

ax.scatter(prob2d_ex3.locations[i, :, 0], prob2d_ex3.locations[i, :, 1],
           marker='*', color='black', s=150)
for pnts, srfnrms, center, radius, integ in zip(points[i],
                                                surfacenorms[i], vols['centers'][i], vols['radii'][i], integs[i]):
    ax.scatter(pnts[:, 0], pnts[:, 1], marker='o',
               color=cmap(1.0-cnorm(integ.item())), s=1)
    ax.quiver(pnts[:, 0], pnts[:, 1], srfnrms[:, 0],
              srfnrms[:, 1], width=0.002)
ax.set_aspect('equal', adjustable='box')

fig

### Visualizing the Unit Cube to Unit Sphere Transformation

In [ ]:
dim = 3
n_bch, n_v, n = 1, 1, 100

ex_device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
ex_tchdevice = torch.device(ex_device)
ex_tchdtype = torch.double

rng = BatchRNG(shape=(n_bch,), lib='torch', device=ex_tchdevice, dtype=ex_tchdtype,
               unif_cache_cols=1000, norm_cache_cols=5000)
rng.seed(np.broadcast_to(12345+np.arange(n_bch), rng.shape))


ex_cube2sphr = Cube2Sphere(n_cdfint=10000, dim=dim, tch_device=ex_tchdevice,
    tch_dtype=ex_tchdtype)

# Step 1: Sampling uniform numbers
#   Option 1: we can use truly random numbers in the [0,1] intervals.
ex_unifs = rng.uniform((n_bch, n_v, n, dim-1))
assert ex_unifs.shape == (n_bch, n_v, n, dim-1)

#   Option 2: we could pass a meshgrid as uniform samples.
n_droot = int(n**(1.0/(dim-1)))
assert n == n_droot ** (dim-1)
u1d = ex_cube2sphr.tch_exlinspace(0, 1, n_droot)
ex_unifs_ = torch.cartesian_prod(*([u1d] * (dim-1))).reshape(n, dim-1)
assert ex_unifs_.shape == (n, dim-1)
ex_unifs = ex_unifs_.reshape(1, 1, n, dim-1).expand(n_bch, n_v, n, dim-1)
assert ex_unifs.shape == (n_bch, n_v, n, dim-1)

# Applying the uniform cube to uniform sphere transformation
sphr_x = ex_cube2sphr(ex_unifs)
assert sphr_x.shape == (n_bch, n_v, n, dim)

In [ ]:
fig = plt.figure(dpi=36)
ax = fig.add_subplot(projection='3d')

x_np = sphr_x.reshape(n_bch * n_v * n, dim).detach().cpu().numpy()
ax.scatter(x_np[:, 2], x_np[:, 1], x_np[:, 0], marker='o', s=20)

ax.grid(False)
ax.set_box_aspect((2, 2, 2))
ax.set_xticks([-1.0, 0.0, 1.0])
ax.set_xticklabels([-1.0, 0.0, 1.0])
ax.set_yticks([]); ax.set_yticklabels([])
ax.set_zticks([]); ax.set_zticklabels([])
for saxis in [ax.xaxis, ax.yaxis, ax.zaxis]:
    saxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    saxis.line.set_color((1.0, 1.0, 1.0, 0.0))
ax.set_xlim3d(-1, 1); ax.set_ylim3d(-1, 1); ax.set_zlim3d(-1, 1)
ax.quiver(-1.0, -1.0, -1.0, 2.4, 0.0, 0.0, color='k', lw=3, arrow_length_ratio=0.1)
ax.quiver(-1.0, -1.0, -1.0, 0.0, 2.4, 0.0, color='k', lw=3, arrow_length_ratio=0.1)
ax.quiver(-1.0, -1.0, -1.0, 0.0, 0.0, 2.4, color='k', lw=3, arrow_length_ratio=0.1)

ax.tick_params(axis='both', which='major', pad=-5)
uu, vv = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
ax.plot_wireframe(np.cos(uu)*np.sin(vv), np.sin(uu)*np.sin(vv), np.cos(vv), color="black", lw=0.1)

fig

## Utility Functions for Sanity Checks

In [ ]:
#########################################################
########### Sanity Checking Utility Functions ###########
#########################################################

msg_bcast = '{} should be np broadcastable to {}={}. '
msg_bcast += 'However, it has an inferred shape of {}.'


def get_arr(name, trgshp_str, trns_opts):
    """
    Gets a list of values, and checks if it is broadcastable to a 
    target shape. If the shape does not match, it will raise a proper
    assertion error with a meaninful message. The output is a numpy 
    array that is guaranteed to be broadcastable to the target shape.

    Parameters
    ----------
    name: (str) name of the option / hyper-parameter.

    trgshp_str: (str) the target shape elements representation. Must be a 
        valid python expression where the needed elements .

    trns_opts: (dict) a dictionary containing the variables needed 
        for the string to list translation of val.

    Key Variables
    -------------
    `val = trns_opts[name]`: (list or str) list of values read 
        from the config file. If a string is provided, python's 
        `eval` function will be used to translate it into a list.
        
    `trg_shape = eval_formula(trgshp_str, trns_opts)`: (tuple) 
        the target shape.
    
    Output
    ----------
    val_np: (np.array) the numpy array of val. 
    """
    msg_ =  f'"{name}" must be in trns_opts but it isnt: {trns_opts}'
    assert name in trns_opts, msg_
    val = trns_opts[name]
    
    if isinstance(val, str):
        val_list = eval_formula(val, trns_opts)
    else:
        val_list = val
    val_np = np.array(val_list)
    src_shape = val_np.shape
    trg_shape = eval_formula(trgshp_str, trns_opts)
    msg_ = msg_bcast.format(name, trgshp_str, trg_shape, src_shape)

    assert len(val_np.shape) == len(trg_shape), msg_

    is_bcastble = all((x == y or x == 1 or y == 1) for x, y in
                      zip(src_shape, trg_shape))
    assert is_bcastble, msg_

    return val_np


def eval_formula(formula, variables):
    """
    Gets a string formula and uses the `eval` function of python to  
    translate it into a python variable. The necessary variables for 
    translation are provided through the `variables` argument.

    Parameters
    ----------
    formula (str): a string that can be passed to `eval`.
        Example: "[np.sqrt(dim), 'a', None]"

    variables (dict): a dictionary of variables used in the formula.
        Example: {"dim": 4}

    Output
    ------
    pyobj (object): the translated formula into a python object
        Example: [2.0, 'a', None]

    """
    locals().update(variables)
    pyobj = eval(formula)
    return pyobj


def chck_dstrargs(opt, cfgdict, dstr2args, opt2req, parnt_optdstr=None):
    """
    Checks if the distribution arguments are provided correctly. Works 
    with hirarchical models through recursive applications. Proper error 
    messages are displayed if one of the checks fails.

    Parameters
    ----------
    opt: (str) the option name.

    cfgdict: (dict) the config dictionary.

    dstr2args: (dict) a mapping between distribution and their 
        required arguments.
        
    opt2req: (dict) required arguments for an option itself, not 
        necessarily required by the option's distribution.
    """
    opt_dstr = cfgdict.get(f'{opt}/dstr', 'fixed')

    msg_ = f'Unknown {opt}_dstr: it should be one of {list(dstr2args.keys())}'
    assert opt_dstr in dstr2args, msg_

    opt2req = dict() if opt2req is None else opt2req
    optreqs = opt2req.get(opt, tuple())
    must_spec = list(dstr2args[opt_dstr]) + list(optreqs)
    avid_spec = list(chain.from_iterable(
        v for k, v in dstr2args.items() if k != opt_dstr))
    avid_spec = [k for k in avid_spec if k not in must_spec]

    if opt_dstr == 'fixed':
        # To avoid infinite recursive calls, we should end this here.
        msg_ = f'"{opt}" must be specified.'
        if parnt_optdstr is not None:
            parnt_opt, parnt_dstr = parnt_optdstr
            msg_ += f'"{parnt_opt}" was specified as "{parnt_dstr}", and'
        msg_ += f' "{opt}" was specified as "{opt_dstr}".'
        if len(optreqs) > 0:
            msg_ += f' Also, "{opt}" requires "{optreqs}" to be specified.'
        opt_val = cfgdict.get(opt, None)
        assert opt_val is not None, msg_
    else:
        for arg in must_spec:
            opt_arg = f'{opt}{arg}'
            chck_dstrargs(opt_arg, cfgdict, dstr2args, opt2req, (opt, opt_dstr))

    for arg in avid_spec:
        opt_arg = f'{opt}{arg}'
        opt_arg_val = cfgdict.get(opt_arg, None)
        msg_ = f'"{opt_arg}" should not be specified, since "{opt}" '
        msg_ += f'appears to follow the "{opt_dstr}" distribution.'
        assert opt_arg_val is None, msg_



## JSON Config Loading and Preprocessing

In [ ]:
json_cfgpath = f'../configs/01_poisson/20_hindim.yml'
! rm -rf "./23_poisson/results/20_hindim.h5"
! rm -rf "./23_poisson/storage/20_hindim"
if json_cfgpath.endswith('.json'):
    with open(json_cfgpath, 'r') as fp:
        json_cfgdict = json.load(fp, object_pairs_hook=odict)
elif json_cfgpath.endswith('.yml'):
    with open(json_cfgpath, "r") as fp:
        json_cfgdict = odict(yaml.safe_load(fp))
else:
    raise RuntimeError(f'unknown config extension: {json_cfgpath}')
json_cfgdict['io/config_id'] = '20_hindim'
json_cfgdict['io/results_dir'] = './23_poisson/results'
json_cfgdict['io/storage_dir'] = './23_poisson/storage'
json_cfgdict['io/tch/device'] = 'cuda:0'

all_cfgdicts = preproc_cfgdict(json_cfgdict)
cfg_dict_input = all_cfgdicts[6]

## Retrieving Config Variables

In [ ]:
    cfg_dict = cfg_dict_input.copy()

    #########################################################
    #################### Ignored Options ####################
    #########################################################
    cfgdesc = cfg_dict.pop('desc', None)
    cfgdate = cfg_dict.pop('date', None)

    #########################################################
    ################### Mandatory Options ###################
    #########################################################
    prob_type = cfg_dict.pop('problem')
    rng_seed_list = cfg_dict.pop('rng_seed/list')
    dim = cfg_dict.pop('dim')

    n_srf = cfg_dict.pop('vol/n')
    n_srfpts_mdl = cfg_dict.pop('srfpts/n/mdl')
    n_srfpts_trg = cfg_dict.pop('srfpts/n/trg')
    do_dblsampling = cfg_dict.pop('srfpts/dblsmpl')
    
    do_bootstrap = cfg_dict.pop('trg/btstrp')
    if do_bootstrap:
        tau = cfg_dict.pop('trg/tau')
        w_trgreg = cfg_dict.pop('trg/reg/w')
    else:
        w_trgreg = 0.0
    w_trg = cfg_dict.pop('trg/w', None)

    opt_type = cfg_dict.pop('opt/dstr')
    n_epochs = cfg_dict.pop('opt/epoch')
    lr = cfg_dict.pop('opt/lr')

In [ ]:
    #########################################################
    ############### Surface Sampling Options ################
    #########################################################
    do_detspacing = cfg_dict.pop('srfpts/detspc', None)
    detspc_cfg = dict()
    if do_detspacing is None:
        pass
    elif do_detspacing == True:
        detspc_cfg['srfpts/trnsfrm/dstr'] = 'cube2sphr'
        detspc_cfg['srfpts/trnsfrm/n_cdfint'] = int(1e6)
        detspc_cfg['srfpts/samp/dstr'] = 'grid'
        detspc_cfg['srfpts/samp/shflpts'] = True
    elif do_detspacing == False:
        detspc_cfg['srfpts/trnsfrm/dstr'] = 'normscale'
        detspc_cfg['srfpts/samp/dstr'] = 'rng'
        detspc_cfg['srfpts/samp/shflpts'] = True
    else:
        msg_ = f'srfpts/detspc="{do_detspacing}" not implemented yet!'
        raise ValueError(msg_)

    for key in detspc_cfg:
        assert key not in cfg_dict, f'cannot specify "{key}" with "srfpts/detspc"'
    cfg_dict.update(detspc_cfg)
    cfg_dict_input.update(detspc_cfg)

    srf_tnsfrm = cfg_dict.pop('srfpts/trnsfrm/dstr')
    if srf_tnsfrm == 'cube2sphr':
        n_cdfint = cfg_dict.pop('srfpts/trnsfrm/n_cdfint')
    elif srf_tnsfrm == 'normscale':
        pass
    else:
        msg_ = f'srfpts/trnsfrm/dstr="{srf_tnsfrm}" not defined!'
        raise ValueError(msg_)
    
    srf_samp = cfg_dict.pop('srfpts/samp/dstr')
    srf_shflpts = cfg_dict.pop('srfpts/samp/shflpts')
    if srf_samp == 'quad':
        quad_order = cfg_dict.pop('srfpts/samp/order')
        quad_rule = cfg_dict.pop('srfpts/samp/rule')
        quad_sprs = cfg_dict.pop('srfpts/samp/sparse')
        quad_rcuralg = cfg_dict.pop('srfpts/samp/rcuralg')
    elif srf_samp == 'qmc':
        qmc_rule = cfg_dict.pop('srfpts/samp/rule')
        qmc_antithetic = cfg_dict.pop('srfpts/samp/antithetic')
    elif srf_samp in ('grid', 'rng'):
        pass
    else:
        msg_ = f'srfpts/samp/dstr="{srf_samp}" not defined!'
        raise ValueError(msg_)

In [ ]:
    #########################################################
    ################## Neural Spec Options ##################
    #########################################################
    nn_dstr = cfg_dict.pop('nn/dstr')
    if nn_dstr == 'mlp':
        nn_width = cfg_dict.pop('nn/width')
        nn_hidden = cfg_dict.pop('nn/hidden')
        nn_act = cfg_dict.pop('nn/act')
    else:
        msg_ = f'nn/dstr="{nn_dstr}" not defined!'
        raise ValueError(msg_)

    #########################################################
    ############## Charge Distribution Options ##############
    #########################################################
    chrg_dstr = cfg_dict.pop('chrg/dstr')
    chrg_n = cfg_dict.pop('chrg/n')

    chrg_w_dstr = cfg_dict.pop('chrg/w/dstr', 'fixed')
    if chrg_w_dstr == 'fixed':
        chrg_w_ = cfg_dict.pop('chrg/w')
    elif chrg_w_dstr == 'uniform':
        chrg_w_low_ = cfg_dict.pop('chrg/w/low', None)
        chrg_w_high_ = cfg_dict.pop('chrg/w/high', None)
    elif chrg_w_dstr == 'normal':
        chrg_w_loc_ = cfg_dict.pop('chrg/w/loc', None)
        chrg_w_scale_ = cfg_dict.pop('chrg/w/scale', None)
    else:
        msg_ = f'chrg/w/dstr="{chrg_w_dstr}" not defined!'
        raise ValueError(msg_)

    chrg_mu_dstr = cfg_dict.pop('chrg/mu/dstr', 'fixed')
    if chrg_mu_dstr == 'fixed':
        chrg_mu_ = cfg_dict.pop('chrg/mu')
    elif chrg_mu_dstr == 'uniform':
        chrg_mu_low_ = cfg_dict.pop('chrg/mu/low')
        chrg_mu_high_ = cfg_dict.pop('chrg/mu/high')
    elif chrg_mu_dstr == 'normal':
        chrg_mu_loc_ = cfg_dict.pop('chrg/mu/loc')
        chrg_mu_scale_ = cfg_dict.pop('chrg/mu/scale')
    elif chrg_mu_dstr == 'ball':
        chrg_mu_c_ = cfg_dict.pop('chrg/mu/c')
        chrg_mu_r_ = cfg_dict.pop('chrg/mu/r')
    else:
        msg_ = f'chrg/mu/dstr="{chrg_mu_dstr}" not defined!'
        raise ValueError(msg_)
    
    #########################################################
    ############### Surface Sampling Options ################
    #########################################################
    vol_dstr = cfg_dict.pop('vol/dstr')

    vol_c_dstr = cfg_dict.pop('vol/c/dstr', 'fixed')
    if vol_c_dstr == 'fixed':
        vol_c_ = cfg_dict.pop('vol/c')
    elif vol_c_dstr == 'uniform':
        vol_c_low_ = cfg_dict.pop('vol/c/low')
        vol_c_high_ = cfg_dict.pop('vol/c/high')
    elif vol_c_dstr == 'normal':
        vol_c_loc_ = cfg_dict.pop('vol/c/loc')
        vol_c_scale_ = cfg_dict.pop('vol/c/scale')
    elif vol_c_dstr == 'ball':
        vol_c_c_ = cfg_dict.pop('vol/c/c')
        vol_c_r_ = cfg_dict.pop('vol/c/r')
    else:
        msg_ = f'vol/c/dstr="{vol_c_dstr}" not defined!'
        raise ValueError(msg_)

    vol_r_dstr = cfg_dict.pop('vol/r/dstr', 'fixed')
    if vol_r_dstr == 'fixed':
        vol_r_ = cfg_dict.pop('vol/r')
    elif vol_r_dstr in ('uniform', 'unifdpow'):
        vol_r_low_ = cfg_dict.pop('vol/r/low')
        vol_r_high_ = cfg_dict.pop('vol/r/high')
    else:
        msg_ = f'vol/r/dstr="{vol_r_dstr}" not defined!'
        raise ValueError(msg_)

    #########################################################
    ############## Initial Condition Options ###############
    #########################################################
    ic_dstr = cfg_dict.pop('ic/dstr', None)
    if ic_dstr in ('sphere', 'trnvol'):
        w_ic = cfg_dict.pop('ic/w')
        ic_bpp = cfg_dict.pop('ic/bpp')
        ic_n = cfg_dict.pop('ic/n')
        ic_frq = cfg_dict.pop('ic/frq')
        ic_bs = cfg_dict.pop('ic/bs')
        ic_needsampling = True
    elif ic_dstr in ('trnsrf',):
        w_ic = cfg_dict.pop('ic/w')
        ic_bpp = cfg_dict.pop('ic/bpp')
        ic_n = n_srf * n_srfpts_mdl
        ic_frq = cfg_dict.pop('ic/frq')
        ic_bs = ic_n
        ic_needsampling = False
    elif ic_dstr is None:
        ic_needsampling = False
        ic_frq = 1
        w_ic, ic_bpp = 0, 'all'
    else:
        msg_ = f'ic/dstr={ic_dstr} not defined'
        raise ValueError(msg_)

    if ic_dstr == 'sphere':
        ic_c_dstr = cfg_dict.pop('ic/c/dstr', 'fixed')
        if ic_c_dstr == 'fixed':
            ic_c_ = cfg_dict.pop('ic/c')
        elif ic_c_dstr == 'uniform':
            ic_c_low_ = cfg_dict.pop('ic/c/low')
            ic_c_high_ = cfg_dict.pop('ic/c/high')
        elif ic_c_dstr == 'normal':
            ic_c_loc_ = cfg_dict.pop('ic/c/loc')
            ic_c_scale_ = cfg_dict.pop('ic/c/scale')
        else:
            msg_ = f'ic/c/dstr="{ic_c_dstr}" not defined!'
            raise ValueError(msg_)

        ic_r_dstr = cfg_dict.pop('ic/r/dstr', 'fixed')
        if ic_r_dstr == 'fixed':
            ic_r_ = cfg_dict.pop('ic/r')
        elif ic_r_dstr in ('uniform', 'unifdpow'):
            ic_r_low_ = cfg_dict.pop('ic/r/low')
            ic_r_high_ = cfg_dict.pop('ic/r/high')
        else:
            msg_ = f'ic/r/dstr="{ic_r_dstr}" not defined!'
            raise ValueError(msg_)
    elif ic_dstr in ('trnsrf', 'trnvol', None):
        pass
    else:
        msg_ = f'ic/dstr="{ic_dstr}" not defined!'
        raise ValueError(msg_)

    #########################################################
    ########### Evaluation Point Sampling Options ###########
    #########################################################
    eid_list_dup = [opt.split('/')[1] for opt in cfg_dict
                    if opt.startswith('eval/')]
    eid_list = list(odict.fromkeys(eid_list_dup))
    evalcfgs = odict()
    for eid in eid_list:
        evalcfgs[eid] = odict()
        cfgopts = list(cfg_dict.keys())
        for opt in cfgopts:
            prfx = f'eval/{eid}/'
            if opt.startswith(prfx):
                optn = opt[len(prfx):]
                optv = cfg_dict.pop(opt)
                evalcfgs[eid][optn] = optv

    #########################################################
    ################# I/O Logistics Options #################
    #########################################################
    config_id = cfg_dict.pop('io/config_id')
    results_dir = cfg_dict.pop('io/results_dir')
    storage_dir = cfg_dict.pop('io/storage_dir', None)
    io_avgfrq = cfg_dict.pop('io/avg/frq')
    ioflsh_period = cfg_dict.pop('io/flush/frq')
    chkpnt_period = cfg_dict.pop('io/ckpt/frq')
    device_name = cfg_dict.pop('io/tch/device')
    dtype_name = cfg_dict.pop('io/tch/dtype')
    iomon_period = cfg_dict.pop('io/mon/frq')
    io_cmprssnlvl = cfg_dict.pop('io/cmprssn_lvl')
    eval_bs = cfg_dict.pop('io/eval/bs', None)

    dtnow = datetime.datetime.now().isoformat(timespec='seconds')
    hostname = socket.gethostname()
    commit_hash = get_git_commit()
    cfg_tree = '/'.join(config_id.split('/')[:-1])
    cfg_name = config_id.split('/')[-1]
    #########################################################
    ##################### Sanity Checks #####################
    #########################################################

    # Making sure the specified option distributions are implemented.
    fixed_opts = ['desc', 'date', 'rng_seed/list', 'problem',
                  'dim', 'vol/n',  'srfpts/n/mdl', 'srfpts/n/trg', 
                  'srfpts/detspc', 'srfpts/dblsmpl', 'trg/btstrp', 
                  'trg/w', 'trg/tau', 'opt/lr', 'opt/epoch',
                  'srfpts/trnsfrm/n_cdfint', 'srfpts/samp/shflpts', 
                  'srfpts/samp/order', 'srfpts/samp/rule', 'srfpts/samp/sparse', 
                  'srfpts/samp/rcuralg', 'srfpts/samp/antithetic']

    opt2availdstrs = {**{opt: ('fixed',) for opt in fixed_opts},
        'chrg': ('dmm',), 'chrg/n': ('fixed',), 'chrg/w': ('fixed',), 
        'chrg/mu': ('fixed', 'uniform', 'normal', 'ball'),
        'vol': ('ball',), 'vol/c': ('uniform', 'ball', 'normal'), 
        'vol/r': ('uniform', 'unifdpow'), 
        'ic': ('sphere', 'trnsrf', 'trnvol', 'fixed'), 
        'ic/c': ('fixed',), 'nn': ('mlp',), 'ic/r': ('fixed',), 
        'vol/c/low': ('fixed',), 'vol/c/high': ('fixed',),
        'vol/c/loc': ('fixed',), 'vol/c/scale': ('fixed',),
        'vol/c/c': ('fixed',),   'vol/c/r': ('fixed',),
        'vol/r/low': ('fixed',), 'vol/r/high': ('fixed',),
        'srfpts/trnsfrm': ('cube2sphr', 'normscale'),
        'srfpts/samp': ('quad', 'qmc', 'rng', 'grid'),
        **{f'eval/{eid}': ('uniform', 'grid', 'ball', 'trnvol')
            for eid in eid_list}}

    for opt, avail_dstrs in opt2availdstrs.items():
        opt_dstr = cfg_dict_input.get(f'{opt}/dstr', 'fixed')
        msg_  = f'"{opt}" cannot follow the "{opt_dstr}" distribution or type '
        msg_ += f' since it is not implemented or available at least yet. The '
        msg_ += f'only available options for "{opt}" are {avail_dstrs}.'
        assert opt_dstr in avail_dstrs, msg_

    # Making sure no other options are left unused.
    if len(cfg_dict) > 0:
        msg_ = f'The following settings were left unused:\n'
        for key, val in cfg_dict.items():
            msg_ += f'  {key}: {val}'
        raise RuntimeError(msg_)

    # Making sure that all "*_dstr" options are valid
    dstr2args = {'uniform':  ('/low', '/high'),
                 'unifdpow': ('/low', '/high'),
                 'normal':   ('/loc', '/scale'),
                 'dmm':      ('/n', '/w', '/mu'),
                 'ball':     ('/c', '/r'),
                 'sphere':   ('/c', '/r'),
                 'fixed':    ('',),
                 'trnvol':   (),
                 'trnsrf':   ()}

    key2req = {'vol': ('/n',)}
    if ic_dstr is not None:
        key2req['ic'] = (*key2req.get('ic', []), '/w')
    if ic_needsampling: 
        key2req['ic'] = (*key2req.get('ic', []), '/n', '/frq')

    for key in ['chrg', 'vol']:
        if key in key2req:
            chck_dstrargs(key, cfg_dict_input, dstr2args, key2req)

    edstr2args = {'uniform': ('/low', '/high', '/n', '/frq'),
                  'grid':    ('/low', '/high', '/n', '/frq'),
                  'ball':    ('/c', '/r', '/n', '/frq'),
                  'fixed':   ('', '/n', '/frq'),
                  'trnvol':  ('/n', '/frq')}
    for eid in eid_list:
        chck_dstrargs(f'eval/{eid}', cfg_dict_input, 
            edstr2args, None)

## Problem Objects Construction 

In [ ]:
    # Derived options and assertions
    n_points = n_srfpts_mdl + n_srfpts_trg

    assert not (do_dblsampling) or (n_srfpts_trg > 1)
    if w_trg is None:
        w_trg = n_srfpts_trg / n_points
    assert not (n_srfpts_mdl == 0) or (w_trg == 1.0)
    n_rsdls = 2 if do_dblsampling else 1

    if eval_bs is None:
        eval_bs = max(n_srfpts_mdl, n_srfpts_trg) * n_srf
        
    #########################################################
    ########### I/O-Related Options and Operations ##########
    #########################################################

    name2dtype = dict(float64=torch.double,
                      float32=torch.float32,
                      float16=torch.float16)
    tch_device = torch.device(device_name)
    tch_dtype = name2dtype[dtype_name]

    tch_dvcmdl = device_name
    if device_name.startswith('cuda'):
        tch_dvcmdl = torch.cuda.get_device_name(tch_device)

    # Reserving 15.596 GB of memory for later usage
    # t_gpumem = torch.cuda.get_device_properties(tch_device).total_memory
    # tdt_elsize = torch.tensor(1).to(tch_device, tch_dtype).element_size()
    # nuslss = int((0.90 * t_gpumem) / tdt_elsize)
    # useless_tensor = torch.empty((nuslss,), device=tch_device, dtype=tch_dtype)
    # del useless_tensor
    
    msg_ = f'"io/mon/frq" % "io/avg/frq" != 0'
    assert iomon_period % io_avgfrq == 0, msg_
    msg_ = f'"io/ckpt/frq" % "io/avg/frq" != 0'
    assert chkpnt_period % io_avgfrq == 0, msg_
    
    do_logtb = storage_dir is not None
    do_profile = storage_dir is not None
    do_tchsave = storage_dir is not None
    
    assert not(do_logtb) or (storage_dir is not None)
    assert not(do_profile) or (storage_dir is not None)
    assert not(do_tchsave) or (storage_dir is not None)

    #########################################################
    ########### Constructing the Batch RNG Object ###########
    #########################################################
    n_seeds = len(rng_seed_list)
    rng_seeds = np.array(rng_seed_list)
    rng = BatchRNG(shape=(n_seeds,), lib='torch',
                   device=tch_device, dtype=tch_dtype,
                   unif_cache_cols=1_000_000,
                   norm_cache_cols=5_000_000)
    rng.seed(np.broadcast_to(rng_seeds, rng.shape))
    erng = rng

    #########################################################
    ########## Defining the Poisson Problem Object ##########
    #########################################################
    assert prob_type == 'poisson'

    msg_ = f'chrg_dstr = {chrg_dstr} is not available/implemented.'
    assert chrg_dstr in ('dmm',), msg_

    trns_opts = dict(dim=dim, chrg_n=chrg_n, sqrt=np.sqrt)

    # The poisson delta charge weights
    if chrg_w_dstr == 'fixed':
        chrg_w_0 = get_arr('chrg_w', '(chrg_n,)', 
            {**trns_opts, 'chrg_w': chrg_w_})
        chrg_w = np.broadcast_to(chrg_w_0[None, ...],
                                 (n_seeds, chrg_n)).copy()
        assert chrg_w.shape == (n_seeds, chrg_n)
    else:
        raise ValueError(f'chrg_w_dstr={chrg_w_dstr} '
                         'not implemented.')

    # The poisson delta charge locations
    if chrg_mu_dstr == 'fixed':
        chrg_mu_0 = get_arr('chrg_mu', '(chrg_n, dim)', 
            {**trns_opts, 'chrg_mu': chrg_mu_})
        chrg_mu = np.broadcast_to(chrg_mu_0[None, ...],
                                  (n_seeds, chrg_n, dim)).copy()
        assert chrg_mu.shape == (n_seeds, chrg_n, dim)
    elif chrg_mu_dstr == 'uniform':
        chrg_mu_low_0 =  get_arr('chrg_mu_low', '(chrg_n, dim)', 
            {**trns_opts, 'chrg_mu_low': chrg_mu_low_})
        chrg_mu_low = np.broadcast_to(chrg_mu_low_0[None, ...],
            (n_seeds, chrg_n, dim)).copy()
        assert chrg_mu_low.shape == (n_seeds, chrg_n, dim)
        
        chrg_mu_high_0 =  get_arr('chrg_mu_high', '(chrg_n, dim)', 
            {**trns_opts, 'chrg_mu_high': chrg_mu_high_})
        chrg_mu_high = np.broadcast_to(chrg_mu_high_0[None, ...],
            (n_seeds, chrg_n, dim)).copy()
        assert chrg_mu_high.shape == (n_seeds, chrg_n, dim)
        
        rnds = rng.uniform((n_seeds, chrg_n, dim)).detach().cpu().numpy()
        chrg_mu = chrg_mu_low + rnds * (chrg_mu_high - chrg_mu_low)
        assert chrg_mu.shape == (n_seeds, chrg_n, dim)
    elif chrg_mu_dstr == 'normal':
        chrg_mu_loc_0 =  get_arr('chrg_mu_loc', '(chrg_n, dim)', 
            {**trns_opts, 'chrg_mu_loc': chrg_mu_loc_})
        chrg_mu_loc = np.broadcast_to(chrg_mu_loc_0[None, ...],
            (n_seeds, chrg_n, dim)).copy()
        assert chrg_mu_loc.shape == (n_seeds, chrg_n, dim)
        
        chrg_mu_scale_0 =  get_arr('chrg_mu_scale', '(chrg_n,)', 
            {**trns_opts, 'chrg_mu_scale': chrg_mu_scale_})
        chrg_mu_scale = np.broadcast_to(chrg_mu_scale_0[None, ...],
            (n_seeds, chrg_n)).copy()
        assert chrg_mu_scale.shape == (n_seeds, chrg_n)
        
        rnds = rng.normal((n_seeds, chrg_n, dim)).detach().cpu().numpy()
        chrg_mu = chrg_mu_loc + rnds * chrg_mu_scale.reshape(n_seeds, chrg_n, 1)
        assert chrg_mu.shape == (n_seeds, chrg_n, dim)
    elif chrg_mu_dstr == 'ball':
        chrg_mu_c_0 =  get_arr('chrg_mu_c', '(chrg_n, dim)', 
            {**trns_opts, 'chrg_mu_c': chrg_mu_c_})
        chrg_mu_c = np.broadcast_to(chrg_mu_c_0[None, ...],
            (n_seeds, chrg_n, dim)).copy()
        assert chrg_mu_c.shape == (n_seeds, chrg_n, dim)
        
        chrg_mu_r_0 =  get_arr('chrg_mu_r', '(chrg_n,)', 
            {**trns_opts, 'chrg_mu_r': chrg_mu_r_})
        chrg_mu_r = np.broadcast_to(chrg_mu_r_0[None, ...],
            (n_seeds, chrg_n)).copy()
        assert chrg_mu_r.shape == (n_seeds, chrg_n)
        
        rnds1 = rng.normal((n_seeds, chrg_n, dim)).detach().cpu().numpy()
        rnds1_tilde = rnds1 / np.sqrt((rnds1**2).sum(axis=-1, keepdims=True))
        assert rnds1_tilde.shape == (n_seeds, chrg_n, dim)
        
        rnds2 = rng.uniform((n_seeds, chrg_n)).detach().cpu().numpy()
        rnds2_tilde = rnds2 ** (1.0 / dim)
        assert rnds2_tilde.shape == (n_seeds, chrg_n)
        
        rnds3_tilde = (chrg_mu_r * rnds2_tilde).reshape(n_seeds, chrg_n, 1)
        assert rnds3_tilde.shape == (n_seeds, chrg_n, 1)
        
        chrg_mu = chrg_mu_c + rnds1_tilde * rnds3_tilde
        assert chrg_mu.shape == (n_seeds, chrg_n, dim)
    else:
        raise ValueError(f'chrg_mu_dstr={chrg_mu_dstr} '
                         'not implemented.')

    # Defining the problem object
    problem = DeltaProblem(weights=chrg_w, locations=chrg_mu,
        tch_device=tch_device, tch_dtype=tch_dtype)
    
    #########################################################
    ####### Defining the Initial Condition Parameters #######
    #########################################################
    msg_ = f'ic/dstr = {ic_dstr} is not available/implemented.'
    assert ic_dstr in ('sphere', 'trnsrf', None), msg_
    msg_ = '"ic/bpp" must be either "bias" or "all".'
    assert ic_bpp in ('bias', 'all', None), msg_

    if ic_dstr == 'sphere':
        if ic_c_dstr == 'fixed':
            ic_c_0_np = get_arr('ic_c', '(dim,)', 
                {**trns_opts, 'ic_c': ic_c_})
            ic_c_np = np.broadcast_to(ic_c_0_np[None, ...], 
                (n_seeds, dim)).copy()
            assert ic_c_np.shape == (n_seeds, dim)
        else:
            raise ValueError(f'ic_c_dstr={ic_c_dstr} '
                            'not implemented.')

        if ic_r_dstr == 'fixed':
            ic_r_0_np = get_arr('ic_r', '()', 
                {**trns_opts, 'ic_r': ic_r_})
            ic_r_np = np.broadcast_to(ic_r_0_np[None, ...], 
                (n_seeds,)).copy()
            assert ic_r_np.shape == (n_seeds,)
        else:
            raise ValueError(f'ic_r_dstr={ic_r_dstr} '
                            'not implemented.')

        with torch.no_grad():
            ic_c = torch.from_numpy(ic_c_np).to(device=tch_device, 
                dtype=tch_dtype).reshape(n_seeds, 1, dim).expand(n_seeds, ic_n, dim)
            assert ic_c.shape == (n_seeds, ic_n, dim)

            ic_r = torch.from_numpy(ic_r_np).to(device=tch_device, 
                dtype=tch_dtype).reshape(n_seeds, 1, 1).expand(n_seeds, ic_n, 1)
            assert ic_r.shape == (n_seeds, ic_n, 1)
    elif ic_dstr in ('trnsrf', 'trnvol', None):
        pass
    else:
        raise ValueError(f'ic_dstr={ic_dstr} not implemented')

    #########################################################
    ########## Defining the Volume Sampling Object ##########
    #########################################################
    msg_ = f'vol/dstr = {vol_dstr} is not available/implemented.'
    assert vol_dstr in ('ball',), msg_

    if vol_c_dstr == 'uniform':
        vol_c_low_0 = get_arr('vol_c_low', '(dim,)', 
            {**trns_opts, 'vol_c_low': vol_c_low_})
        vol_c_low = np.broadcast_to(vol_c_low_0[None, ...],
            (n_seeds, dim)).copy()
        assert vol_c_low.shape == (n_seeds, dim)

        vol_c_high_0 = get_arr('vol_c_high', '(dim,)', 
            {**trns_opts, 'vol_c_high': vol_c_high_})
        vol_c_high = np.broadcast_to(vol_c_high_0[None, ...],
            (n_seeds, dim)).copy()
        assert vol_c_high.shape == (n_seeds, dim)
        
        vol_c_params = dict(low=vol_c_low, high=vol_c_high)
    elif vol_c_dstr == 'normal':
        vol_c_loc_0 = get_arr('vol_c_loc', '(dim,)', 
            {**trns_opts, 'vol_c_loc': vol_c_loc_})
        vol_c_loc = np.broadcast_to(vol_c_loc_0[None, ...],
            (n_seeds, dim)).copy()
        assert vol_c_loc.shape == (n_seeds, dim)

        vol_c_scale_0 = get_arr('vol_c_scale', '()', 
            {**trns_opts, 'vol_c_scale': vol_c_scale_})
        vol_c_scale = np.broadcast_to(vol_c_scale_0[None, ...],
            (n_seeds,)).copy()
        assert vol_c_scale.shape == (n_seeds,)
        
        vol_c_params = dict(loc=vol_c_loc, scale=vol_c_scale)
    elif vol_c_dstr == 'ball':
        vol_c_c_0 = get_arr('vol_c_c', '(dim,)', 
            {**trns_opts, 'vol_c_c': vol_c_c_})
        vol_c_c = np.broadcast_to(vol_c_c_0[None, ...],
            (n_seeds, dim)).copy()
        assert vol_c_c.shape == (n_seeds, dim)

        vol_c_r_0 = get_arr('vol_c_r', '()', 
            {**trns_opts, 'vol_c_r': vol_c_r_})
        vol_c_r = np.broadcast_to(vol_c_r_0[None, ...],
            (n_seeds,)).copy()
        assert vol_c_r.shape == (n_seeds,)
        
        vol_c_params = dict(c=vol_c_c, r=vol_c_r)
    else:
        raise ValueError(f'vol_c_dstr={vol_c_dstr} not implemented.')

    if vol_r_dstr in ('uniform', 'unifdpow'):
        vol_r_low_0 = get_arr('vol_r_low', '()', 
            {**trns_opts, 'vol_r_low': vol_r_low_, 
             'vol_r_high': vol_r_high_})
        vol_r_low = np.broadcast_to(vol_r_low_0[None, ...],
            (n_seeds,)).copy()
        assert vol_r_low.shape == (n_seeds,)

        vol_r_high_0 = get_arr('vol_r_high', '()', 
            {**trns_opts, 'vol_r_low': vol_r_low_, 
             'vol_r_high': vol_r_high_})
        vol_r_high = np.broadcast_to(vol_r_high_0[None, ...],
            (n_seeds,)).copy()
        assert vol_r_high.shape == (n_seeds,)
        
        vol_r_params = dict(low=vol_r_low, high=vol_r_high)
    else:
        raise ValueError(f'vol_r_dstr={vol_r_dstr} not implemented.')

    volsampler = BallSampler(c_dstr=vol_c_dstr, c_params=vol_c_params,
                             r_dstr=vol_r_dstr, r_params=vol_r_params,
                             batch_rng=rng)

In [ ]:
    srfsampler = SphereSampler(batch_rng=rng)
    
    srftnsfrm_params = {'dstr': srf_tnsfrm}
    if srf_tnsfrm == 'cube2sphr':
        cube2sphr = Cube2Sphere(n_cdfint=n_cdfint, dim=dim, 
            tch_device=tch_device, tch_dtype=tch_dtype)
        srftnsfrm_params['cube2sphr'] = cube2sphr
        rv_dim = dim - 1
    elif srf_tnsfrm == 'normscale':
        cube2sphr, rv_dim = None, dim
    else:
        msg_ = f'srf_tnsfrm="{srf_tnsfrm}" not defined!'
        raise ValueError(msg_)
        
    if (srf_tnsfrm == 'cube2sphr') and (srf_samp in ['quad', 'qmc']):
        cpy_dist = chaospy.Iid(chaospy.Uniform(0, 1), dim-1)
    elif (srf_tnsfrm == 'normscale') and (srf_samp in ['quad', 'qmc']):
        cpy_dist = chaospy.Iid(chaospy.Normal(0, 1), dim)
    elif srf_samp in ['quad', 'qmc']:
        msg_ = f'srf_tnsfrm="{srf_tnsfrm}" not defined!'
        raise ValueError(msg_)
    
    if srf_samp == 'quad':
        qx_np, quadw_np = chaospy.generate_quadrature(
            order=quad_order, dist=cpy_dist, rule=quad_rule, 
            recurrence_algorithm=quad_rcuralg, sparse=quad_sprs)
    elif srf_samp == 'qmc':
        cpy_jac = chaospy.J(cpy_dist)
        qx_np = cpy_jac.sample(n_points, rule=qmc_rule, 
            antithetic=qmc_antithetic, include_axis_dim=True)
    elif srf_samp in ['grid', 'rng']:
        pass
    else:
        raise ValueError('not implemented yet')
    
    if (srf_tnsfrm == 'cube2sphr') and (srf_samp in ['quad', 'qmc']):
        assert np.isclose(min(0.0, qx_np.min()), 0.0)
        assert np.isclose(max(1.0, qx_np.max()), 1.0)
        qx_np_ = np.clip(qx_np, 0.0, 1.0)
    elif (srf_tnsfrm == 'normscale') and (srf_samp in ['quad', 'qmc']):
        # Important to avoid all zeros scenario that cannot be normalized
        qx_np_ = qx_np + 1e-12
        assert (np.square(qx_np_).sum(axis=0) > 0.0).all()
    elif srf_samp in ['quad', 'qmc']:
        raise ValueError('not implemented yet')
    
    if srf_samp == 'quad':
        n_quad = quadw_np.size
        msg_ =  f'The number of quadrature points {n_quad} should be the '
        msg_ += f'same as the sum of "srfpts/n/mdl"={n_srfpts_mdl} and '
        msg_ += f'"srfpts/n/trg"={n_srfpts_trg}.'
        assert n_quad == n_points, msg_
        quad_x = torch.from_numpy(qx_np_.T).to(device=tch_device, dtype=tch_dtype)
        assert quad_x.shape == (n_quad, rv_dim)
        quad_w = torch.from_numpy(quadw_np).to(device=tch_device, dtype=tch_dtype) * n_quad
        assert quad_w.shape == (n_quad,)
        srfsamp_params = {'dstr': 'quad', 'x': quad_x, 'w': quad_w}
    elif srf_samp == 'qmc':
        qmc_x = torch.from_numpy(qx_np_.T).to(device=tch_device, dtype=tch_dtype)
        assert qmc_x.shape == (n_points, rv_dim), f'{qmc_x.shape} != {(n_points, rv_dim)}'
        srfsamp_params = {'dstr': 'qmc', 'x': qmc_x}
    elif srf_samp in ['grid', 'rng']:
        srfsamp_params = {'dstr': srf_samp}
    else:
        raise ValueError('not implemented yet')

In [ ]:
    #########################################################
    #### Evaluation Param Tensorization and Sanitization ####
    #########################################################

    # The following evaluates the 'eval/*' options and creates 
    # array parameters for evaluation.
    # The input is mainly the `evalcfgs` dictionary, which holds 
    # some keys and list/string values. The output will be the 
    # `evalprms` dictionary which has the same keys but with 
    # np.array values.
    # --------------
    # Example input: 
    #   evalcfgs = {'ur': {'dstr': 'uniform', 
    #       'low': [0], 
    #       'high': '[sqrt(dim)]'}}
    # --------------
    # Example output
    #   evalprms = {'ur': {'dstr': 'uniform',
    #       'low' : torch.tensor([0]).expand(n_seeds, dim)
    #       'high': torch.tensor(np.sqrt(dim)).expand(n_seeds, dim))}}

    dstr2shapes = {'uniform': {'low':  '(dim,)', 
                               'high': '(dim,)'},
                   'grid':    {'low':  '(dim,)', 
                               'high': '(dim,)'},
                   'ball':    {'c':    '(dim,)', 
                               'r':    '()'    },
                   'trnvol': {}}

    assert all('dstr' in eopts for eopts in evalcfgs.values())
    assert all('frq'  in eopts for eopts in evalcfgs.values())
    assert all('n'    in eopts for eopts in evalcfgs.values())
    evalprms = odict()
    for eid, eopts_ in evalcfgs.items():
        eopts = eopts_.copy()
        eparam = odict()
        for eopt in ('dstr', 'n', 'frq'):
            eparam[eopt] = eopts.pop(eopt)

        edstr = eparam['dstr']
        msg_  = f'Unknown eval "{eid}" dstr -> "{edstr}". '
        msg_ += f'dstr should be one of {dstr2shapes.keys()}.'
        assert edstr in dstr2shapes, msg_

        estore_dflt = (edstr == 'grid')
        eparam['store'] = eopts.pop('store', estore_dflt)

        opts2shape = dstr2shapes[edstr]
        for eopt, eoptshpstr in opts2shape.items():
            # Example: edstr = 'uniform'
            #          eopt = 'low'
            #          eoptshpstr = '(dim,)'
            #          eoptshp = (dim,)
            #          eoptval = "[sqrt(dim)]"
            #          eopt_pnp0 = np.array([sqrt(dim)]*dim)
            #          eopt_pnp0.shape = (dim,)
            #          eopt_pnp = eopt_pnp0.expand(n_seeds, dim)
            #          eopt_pnp.shape = (n_seeds, dim)
            #          eopt_p = torch.from_numpy(eopt_pa0)
            #          eopt_p.shape = (n_seeds, dim)
            msg_  = f'"eval/{eid}/{eopt}" must be fixed and determined.'
            msg_ += f' Hierarchical support not available yet.'
            assert eopt in eopts, msg_

            eoptval = eopts.pop(eopt)
            etrns = {eopt: eoptval, 'dim': dim, 
                     'sqrt': np.sqrt}

            eopt_pnp0 = get_arr(eopt, eoptshpstr, etrns)        
            eoptshp = eval_formula(eoptshpstr, {'dim': dim})
            eopt_pnp = np.broadcast_to(eopt_pnp0[None, ...],
                                       (n_seeds, *eoptshp)).copy()
            assert eopt_pnp.shape == (n_seeds, *eoptshp)
            eopt_pc = torch.from_numpy(eopt_pnp)
            eopt_p = eopt_pc.to(device=tch_device, dtype=tch_dtype)
            assert eopt_p.shape == (n_seeds, *eoptshp)
            eparam[eopt] = eopt_p
        
        if edstr in ('trnvol', 'ball'):
            for eopt in ('rqnts/dstr', 'rqnts/n', 'use_crn',
                         'rx/dstr', 'rx/static', 'rx/r/dstr', 
                         'rx/r/n', 'rx/r/static', 'rx/x/dstr', 
                         'rx/x/static'):
                if eopt in eopts:
                    eparam[eopt] = eopts.pop(eopt)
        else:
            for eopt in ('rx/dstr', 'rx/static'):
                if eopt in eopts:
                    eparam[eopt] = eopts.pop(eopt)

        assert len(eopts) == 0, f'unused eval items left: {eopts}'
        evalprms[eid] = eparam

    #########################################################
    ############# Evaluation Parameter Creation #############
    #########################################################
    for eid, eopts in evalprms.items():
        edstr = eopts['dstr']
        n_evlpnts = eopts['n']
        if edstr == 'uniform':
            e_low_ = eopts['low']
            assert e_low_.shape == (n_seeds, dim)
            e_low = e_low_.unsqueeze(dim=-2)
            assert e_low.shape == (n_seeds, 1, dim)

            e_high_ = eopts['high']
            assert e_high_.shape == (n_seeds, dim)
            e_high = e_high_.unsqueeze(dim=-2)
            assert e_high.shape == (n_seeds, 1, dim)

            e_slope = e_high - e_low
            assert e_slope.shape == (n_seeds, 1, dim)

            eopts['bias'] = e_low
            eopts['slope'] = e_slope
        elif edstr == 'ball':
            e_c_ = eopts['c']
            assert e_c_.shape == (n_seeds, dim)
            e_c = e_c_.unsqueeze(dim=-2).expand(n_seeds, n_evlpnts, dim)
            assert e_c.shape == (n_seeds, n_evlpnts, dim)

            e_r_ = eopts['r']
            assert e_r_.shape == (n_seeds,)

            e_r = e_r_.reshape(n_seeds, 1, 1).expand(n_seeds, n_evlpnts, 1)
            assert e_r.shape == (n_seeds, n_evlpnts, 1)

            eopts['c_xpnd'] = e_c
            eopts['r_xpnd'] = e_r
        elif edstr == 'trnvol':
            pass
        elif edstr == 'grid':
            n_g = eopts['n']
            n_gpd = int(np.ceil(n_g**(1./dim)))
            assert n_g == (n_gpd ** dim)

            elowt = eopts['low']
            assert elowt.shape == (n_seeds, dim)

            ehight = eopts['high']
            assert ehight.shape == (n_seeds, dim)

            assert (elowt[:1] == elowt).all()
            assert (ehight[:1] == ehight).all()

            elow = elowt.cpu().detach().numpy()[0].tolist()
            ehigh = ehight.cpu().detach().numpy()[0].tolist()

            gdict = make_grid(elow, ehigh, dim, n_gpd, 'torch')
            e_pnts_ = gdict['x']
            assert e_pnts_.shape == (n_g, dim)

            e_pnts = e_pnts_.reshape(1, n_g, dim).expand(n_seeds, n_g, dim)
            assert e_pnts.shape == (n_seeds, n_g, dim)

            eopts['pnts'] = e_pnts.to(tch_device, tch_dtype)
            eopts['xi_msh_np'] = gdict['xi_msh_np']
        else:
            raise ValueError(f'"{edstr}" not defined')
        
        if edstr in ('ball', 'trnvol'):
            # `iscrstatic == True` means that the `e_pntcs` and `e_pntrs` must 
            # be sampled from the `(c_xpnd, r_xpnd)` ball using the RNG 
            # dynamically through the training.            
            
            # Legacy config conversion to the latest form
            e_rqntsdstrold_ = eopts.get('rqnts/dstr', None)
            e_usecrn_ = eopts.get('use_crn', None)
            if e_rqntsdstrold_ is None:
                pass
            elif e_rqntsdstrold_ == 'det':
                assert 'rx/dstr' not in eopts
                eopts['rx/dstr'] = 'indep'
                assert 'rx/r/dstr' not in eopts
                eopts['rx/r/dstr'] = 'det'
                eopts['rx/r/n'] = eopts['rqnts/n']
                eopts['rx/x/dstr'] = 'iid'
                eopts['rx/x/static'] = e_usecrn_ not in (False, None)
            elif e_rqntsdstrold_ == 'iid':
                eopts['rx/dstr'] = 'joint'
                eopts['rx/static'] = e_usecrn_ not in (False, None)
            else:
                raise ValueError(f'rqnts/dstr={e_rqntsdstrold_} not implmntd')
            
            e_rxdstr = eopts.get('rx/dstr', 'joint')
            if e_rxdstr == 'joint':
                e_isrxstatic = eopts.get('rx/static', False)
                
                e_iscrstatic, e_isthstatic = False, False
                # Don't worry! The static points will be generated and 
                # memorized in the first epoch. No need to reimplement 
                # all the routines here as well.
            elif e_rxdstr == 'indep':
                erqnt_type = eopts.get('rx/r/dstr')
                if erqnt_type == 'det':
                    e_iscrstatic = True
                elif erqnt_type == 'iid':
                    e_iscrstatic = eopts['rx/r/static']
                else:
                    raise ValueError(f'rx/r/dstr={erqnt_type} not implmntd')
                
                e_thdstr = eopts.get('rx/x/dstr')
                if e_thdstr == 'iid':
                    e_isthstatic = eopts.get('rx/x/static')
                else:
                    raise ValueError(f'rx/r/dstr={e_thdstr} not implmntd')
                
                n_r_ = eopts['rx/r/n']
                n_th_ = n_evlpnts // n_r_
                msg_ == 'to match the random effects, we must have (n_evlpnts mod rqnts/n == 0)'
                assert n_evlpnts % n_r_ == 0, msg_
            else:
                raise ValueError(f'rx/dstr={e_rxdstr} is not implmntd')
            
            eopts['iscrstatic'] = e_iscrstatic
            eopts['isthstatic'] = e_isthstatic
            
            if e_rxdstr == 'joint':
                e_pntcs, e_pntrs, ethtilde = None, None, None
                
            if (e_rxdstr == 'indep') and e_iscrstatic and (edstr == 'trnvol'):
                ####################################################
                ### Step (1): Finding the mean ball-center point ###
                ####################################################
                vol_cdstr = volsampler.c_dstr
                if vol_cdstr == 'ball':
                    e_pntcs_ = volsampler.c_cntr
                    assert e_pntcs_.shape == (n_seeds, 1, dim)
                elif vol_cdstr == 'normal':
                    e_pntcs_ = volsampler.c_loc
                    assert e_pntcs_.shape == (n_seeds, 1, dim)
                elif vol_cdstr == 'uniform':
                    msg_  = f'uniform vol centers are mathematically '
                    msg_ += f'incompatible with static radii evaluations.'
                    raise ValueError(msg_)
                else:
                    msg_ = f'Unknown volume c_dstr={vol_cdstr}'
                    raise ValueError(msg_)
                e_pntcs = e_pntcs_.expand(n_seeds, n_evlpnts, dim)
                assert e_pntcs.shape == (n_seeds, n_evlpnts, dim)
                
                ####################################################
                ## Step (2): Simulating TV points from Volsampler ##
                ####################################################
                n_sim = 10000 if erqnt_type == 'det' else n_r_
                    
                evols_sim = volsampler(n=n_sim)
                assert evols_sim['type'] == 'ball'
                e_pntcssim = evols_sim['centers']
                assert e_pntcssim.shape == (n_seeds, n_sim, dim)
                e_rsim_ = evols_sim['radii']
                assert e_rsim_.shape == (n_seeds, n_sim)
                e_rsim = e_rsim_.unsqueeze(dim=-1)
                assert e_rsim.shape == (n_seeds, n_sim, 1)
                euntrdsim = erng.uniform((n_seeds, n_sim, 1))
                assert euntrdsim.shape == (n_seeds, n_sim, 1)
                untrsim = euntrdsim.pow(1.0 / dim)
                assert untrsim.shape == (n_seeds, n_sim, 1)
                e_pntrssim = untrsim * e_rsim
                assert e_pntrssim.shape == (n_seeds, n_sim, 1)
                
                ethetasim = erng.normal((n_seeds, n_sim, dim))
                assert ethetasim.shape == (n_seeds, n_sim, dim)
                e_thtildesim = ethetasim / ethetasim.norm(dim=-1, keepdim=True)
                assert e_thtildesim.shape == (n_seeds, n_sim, dim)
                e_pntssim = e_pntcssim + e_thtildesim * e_pntrssim
                assert e_pntssim.shape == (n_seeds, n_sim, dim)
                e_pntsnomeansim = e_pntssim - e_pntcs_
                assert e_pntsnomeansim.shape == (n_seeds, n_sim, dim)
                e_realrsim = e_pntsnomeansim.norm(p=2, dim=-1)
                assert e_realrsim.shape == (n_seeds, n_sim)

                if erqnt_type == 'iid':
                    e_pntrs_ = e_realrsim.reshape(n_seeds, n_r_, 1).expand(n_seeds, n_r_, n_th_)
                    assert e_pntrs_.shape == (n_seeds, n_r_, n_th_)
                    
                    e_pntrs = e_pntrs_.reshape(n_seeds, n_evlpnts, 1)
                    assert e_pntrs.shape == (n_seeds, n_evlpnts, 1)
                elif erqnt_type == 'det':
                    ####################################################
                    ####### Step (3): Computing Radius Quantiles #######
                    ####################################################
                    r_qs_ = torch.arange(n_r_, device=tch_device, dtype=tch_dtype) / n_r_ + 1.0 / (2 * n_r_)
                    assert r_qs_.shape == (n_r_,)
                    e_rqnts_ = torch.quantile(e_realrsim, r_qs_, dim=-1)
                    assert e_rqnts_.shape == (n_r_, n_seeds)
                    e_rqnts = e_rqnts_.T
                    assert e_rqnts.shape == (n_seeds, n_r_)
                    
                    e_pntrs_ = e_rqnts.reshape(n_seeds, n_r_, 1).expand(n_seeds, n_r_, n_th_)
                    assert e_pntrs_.shape == (n_seeds, n_r_, n_th_)
                    e_pntrs = e_pntrs_.reshape(n_seeds, n_evlpnts, 1)
                    assert e_pntrs.shape == (n_seeds, n_evlpnts, 1)
                else:
                    raise ValueError(f'erqnt_type={erqnt_type} not implmntd')                
            elif (e_rxdstr == 'indep') and e_iscrstatic and (edstr == 'ball'):
                e_pntcs = eopts['c_xpnd']
                assert e_pntcs.shape == (n_seeds, n_evlpnts, dim)
                
                e_r_ = eopts['r']
                assert e_r_.shape == (n_seeds,)
                if erqnt_type == 'iid':
                    r_qs = erng.uniform((n_seeds, n_r_, 1)).expand(n_seeds, n_r_, n_th_)
                    assert r_qs.shape == (n_seeds, n_r_, n_th_)
                elif erqnt_type == 'det': 
                    r_qs_ = torch.arange(n_r_, device=tch_device, dtype=tch_dtype) / n_r_ + 1.0 / (2 * n_r_)
                    assert r_qs_.shape == (n_r_,)
                    r_qs = r_qs_.reshape(1, n_r_, 1).expand(n_seeds, n_r_, n_th_)
                    assert r_qs.shape == (n_seeds, n_r_, n_th_)
                else:
                    raise ValueError(f'erqnt_type={erqnt_type} not implmntd')
                
                e_pntrs_ = (r_qs ** (1.0 / dim)) * e_r_.reshape(n_seeds, 1, 1)
                assert e_pntrs_.shape == (n_seeds, n_r_, n_th_)
                e_pntrs = e_pntrs_.reshape(n_seeds, n_evlpnts, 1)
                assert e_pntrs.shape == (n_seeds, n_evlpnts, 1)
            elif (e_rxdstr == 'indep') and not(e_iscrstatic):
                e_pntcs, e_pntrs = None, None
            elif (e_rxdstr == 'indep'):
                raise ValueError(f'not implemeted edstr={edstr}')
            
            if (e_rxdstr == 'indep') and e_isthstatic and e_iscrstatic:
                etheta_ = erng.normal((n_seeds, 1, n_th_, dim))
                assert etheta_.shape == (n_seeds, 1, n_th_, dim)
                
                etheta2_ = etheta_.expand(n_seeds, n_r_, n_th_, dim)
                assert etheta2_.shape == (n_seeds, n_r_, n_th_, dim)
                
                etheta = etheta2_.reshape(n_seeds, n_evlpnts, dim)
                assert etheta.shape == (n_seeds, n_evlpnts, dim)

                ethtilde = etheta / etheta.norm(dim=-1, keepdim=True)
                assert ethtilde.shape == (n_seeds, n_evlpnts, dim)
            elif (e_rxdstr == 'indep') and e_isthstatic and not(e_iscrstatic):                
                etheta = erng.normal((n_seeds, n_evlpnts, dim))
                assert etheta.shape == (n_seeds, n_evlpnts, dim)

                ethtilde = etheta / etheta.norm(dim=-1, keepdim=True)
                assert ethtilde.shape == (n_seeds, n_evlpnts, dim)   
            elif (e_rxdstr == 'indep'):
                ethtilde = None
                
            eopts['pnt_c'] = e_pntcs
            eopts['pnt_r'] = e_pntrs
            eopts['thtilde'] = ethtilde

In [ ]:
    #########################################################
    #### Collecting the Config Columns in the Dataframe #####
    #########################################################
    # Identifying the hyper-parameter from etc config columns
    hppats = ['problem', 'dim', 'vol/n', 'srfpts/n/mdl', 
        'srfpts/n/trg',  'srfpts/detspc', 'srfpts/dblsmpl',
        'trg/w', 'trg/btstrp', 'trg/tau', 'trg/reg/w', 'opt/lr', 
        'opt/dstr',  'nn/*', 'chrg/*', 'ic/*', 'vol/*', 'eval/*', 
        'srfpts/trnsfrm/*', 'srfpts/samp/*']
    etcpats = ['desc', 'date', 'opt/epoch', 'rng_seed/list', 'io/*']

    hpopts = [x for pat in hppats for x in 
               fnmatch.filter(cfg_dict_input.keys(), pat)]
    etcopts = [x for pat in etcpats for x in 
               fnmatch.filter(cfg_dict_input.keys(), pat)]

    err_list = []
    for opt in cfg_dict_input:
        if (opt in hpopts) and (opt in etcopts):
            msg_ = f'"{opt}" should both be treated as hp and etc!'
            err_list.append(msg_)
        if (opt not in hpopts) and (opt not in etcopts):
            msg_ = f'"{opt}" is neither hp nor etc!'
            err_list.append(msg_)
    if len(err_list) > 0:
        raise RuntimeError(('\n'+80*'*'+'\n').join(err_list))

    # Converting the list and tuples to strings
    hp_dict_ = odict()
    etc_dict_ = odict()
    for opt, val in cfg_dict_input.items():
        val = cfg_dict_input[opt]
        if isinstance(val, (int, float, str, bool, type(None))):
            srlval = val
        elif isinstance(val, (list, tuple)):
            srlval = repr(val)
        else: 
            msg_  = f'Not sure how to log "{opt}" with '
            msg_ += f'a value type of "{type(val)}"'
            raise RuntimeError(msg_)

        if opt in hpopts:
            hp_dict_[opt] = srlval
        elif opt in etcopts:
            etc_dict_[opt] = srlval
        else:
            raise RuntimeError(f'Not sure how to log "{opt}"')

    # Few exceptions for the etc directory
    etc_dict_['hostname'] = hostname
    etc_dict_['commit'] = commit_hash
    etc_dict_['date/cfg'] = etc_dict_.pop('date')
    etc_dict_['date/run'] = dtnow
    etc_dict_['io/dvc_mdl'] = tch_dvcmdl
    etc_dict_.pop('io/results_dir')
    etc_dict_.pop('io/storage_dir')

    # Repeating the values by n_seeds
    hp_dict = odict()
    for opt, val in hp_dict_.items():
        hp_dict[opt] = [val] * n_seeds
    etc_dict = odict()
    for opt, val in etc_dict_.items():
        etc_dict[opt] = [val] * n_seeds

# Training Loop

In [ ]:
    if results_dir is not None:
        pathlib.Path(os.sep.join([results_dir, cfg_tree])
                     ).mkdir(parents=True, exist_ok=True)
    if storage_dir is not None:
        cfgstrgpnt_dir = os.sep.join([storage_dir, cfg_tree, cfg_name])
        pathlib.Path(cfgstrgpnt_dir).mkdir(parents=True, exist_ok=True)
        strgidx = sum(isdir(f'{cfgstrgpnt_dir}/{x}') for x in os.listdir(cfgstrgpnt_dir))
        dtnow_ = dtnow[2:].replace('-', '').replace(':', '').replace('.', '')
        cfgstrg_dir = f'{cfgstrgpnt_dir}/{strgidx:02d}_{dtnow_}'
        pathlib.Path(cfgstrg_dir).mkdir(parents=True, exist_ok=True)
    if do_logtb:
        if 'tbwriter' in locals():
            tbwriter.close()
        tbwriter = tensorboardX.SummaryWriter(cfgstrg_dir)
    if do_profile:
        profiler = Profiler()
        profiler.start()

In [ ]:
    # Initializing the model
    model = bffnn(dim, nn_width, nn_hidden, nn_act, (n_seeds,), rng)
    if do_bootstrap:
        target = bffnn(dim, nn_width, nn_hidden, nn_act, (n_seeds,), rng)
        target.load_state_dict(model.state_dict())
    else:
        target = model

    # Set the optimizer
    if opt_type == 'adam':
        opt = torch.optim.Adam(model.parameters(), lr)
    elif opt_type == 'sgd':
        opt = torch.optim.SGD(model.parameters(), lr)
    else:
        raise NotImplementedError(f'opt/dstr="{opt_type}" not implmntd')

    # Evaluation tools
    last_perfdict = dict()
    ema = EMA(gamma=0.999, gamma_sq=0.998)
    trn_sttime = time.time()

    # Data writer construction
    hdfpth = None
    if results_dir is not None:
        hdfpth = f'{results_dir}/{cfg_tree}/{cfg_name}.h5'
    avg_history = odict()
    dwriter = DataWriter(flush_period=ioflsh_period*n_seeds, 
                         compression_level=io_cmprssnlvl)

    if storage_dir is not None:
        with plt.style.context('default'):
            figax_list = [plt.subplots(1, 1, figsize=(3.2, 2.5), dpi=100) for _ in range(3)]
            (fig_mdl, ax_mdl), (fig_trg, ax_trg), (fig_gt, ax_gt) = figax_list
            cax_list = [make_axes_locatable(ax).append_axes('right', size='5%', pad=0.05) 
                        for ax in (ax_mdl, ax_trg, ax_gt)]
            cax_mdl, cax_trg, cax_gt = cax_list
        stat_history = defaultdict(list)
        model_history = odict()
        target_history = odict()

In [ ]:
    for epoch in range(n_epochs+1):        
        opt.zero_grad()

        # Sampling the volumes
        volsamps = volsampler(n=n_srf)

        # Sampling the points from the srferes
        srfsamps = srfsampler(volsamps, n_points, trnsfrm_params=srftnsfrm_params, 
            samp_params=srfsamp_params, do_randrots=True, do_shflpts=srf_shflpts)
        points = nn.Parameter(srfsamps['points'])
        weights = srfsamps['weights']
        surfacenorms = srfsamps['normals']
        areas = srfsamps['areas']
        assert points.shape == (n_seeds, n_srf, n_points, dim)
        assert surfacenorms.shape == (n_seeds, n_srf, n_points, dim)
        assert weights.shape == (n_seeds, n_srf, n_points)
        assert areas.shape == (n_seeds, n_srf)

        points_mdl = points[:, :, :n_srfpts_mdl, :]
        assert points_mdl.shape == (n_seeds, n_srf, n_srfpts_mdl, dim)
        points_trg = points[:, :, n_srfpts_mdl:, :]
        assert points_trg.shape == (n_seeds, n_srf, n_srfpts_trg, dim)

        surfacenorms_mdl = surfacenorms[:, :, :n_srfpts_mdl, :]
        assert surfacenorms_mdl.shape == (n_seeds, n_srf, n_srfpts_mdl, dim)
        surfacenorms_trg = surfacenorms[:, :, n_srfpts_mdl:, :]
        assert surfacenorms_trg.shape == (n_seeds, n_srf, n_srfpts_trg, dim)
        
        weights_mdl = weights[:, :, :n_srfpts_mdl]
        assert weights_mdl.shape == (n_seeds, n_srf, n_srfpts_mdl)
        weights_trg = weights[:, :, n_srfpts_mdl:]
        assert weights_trg.shape == (n_seeds, n_srf, n_srfpts_trg)

        # Making surface integral predictions using the reference model
        u_mdl = model(points_mdl)
        assert u_mdl.shape == (n_seeds, n_srf, n_srfpts_mdl, 1)
        nabla_x_u_mdl, = torch.autograd.grad(u_mdl.sum(), [points_mdl],
            grad_outputs=None, retain_graph=True, create_graph=True,
            only_inputs=True, allow_unused=False)
        assert nabla_x_u_mdl.shape == (n_seeds, n_srf, n_srfpts_mdl, dim)
        normprods_mdl = (nabla_x_u_mdl * surfacenorms_mdl).sum(dim=-1)
        assert normprods_mdl.shape == (n_seeds, n_srf, n_srfpts_mdl)
        wnormprods_mdl = normprods_mdl * weights_mdl
        assert wnormprods_mdl.shape == (n_seeds, n_srf, n_srfpts_mdl)
        if n_srfpts_mdl > 0:
            mean_wnormprods_mdl = wnormprods_mdl.mean(dim=-1, keepdim=True)
            assert mean_wnormprods_mdl.shape == (n_seeds, n_srf, 1)
        else:
            mean_wnormprods_mdl = 0.0

        # Making surface integral predictions using the target model
        u_trg = target(points_trg)
        assert u_trg.shape == (n_seeds, n_srf, n_srfpts_trg, 1)
        nabla_x_u_trg, = torch.autograd.grad(u_trg.sum(), [points_trg],
            grad_outputs=None, retain_graph=True, create_graph=not(do_bootstrap),
            only_inputs=True, allow_unused=False)
        assert nabla_x_u_trg.shape == (n_seeds, n_srf, n_srfpts_trg, dim)

        normprods_trg = (nabla_x_u_trg * surfacenorms_trg).sum(dim=-1)
        assert normprods_trg.shape == (n_seeds, n_srf, n_srfpts_trg)
        wnormprods_trg = normprods_trg * weights_trg
        assert wnormprods_trg.shape == (n_seeds, n_srf, n_srfpts_trg)
        if do_dblsampling:
            assert n_rsdls == 2

            mean_wnormprods_trg1 = wnormprods_trg[..., 0::2].mean(
                dim=-1, keepdim=True)
            assert mean_wnormprods_trg1.shape == (n_seeds, n_srf, 1)

            mean_wnormprods_trg2 = wnormprods_trg[..., 1::2].mean(
                dim=-1, keepdim=True)
            assert mean_wnormprods_trg2.shape == (n_seeds, n_srf, 1)

            mean_wnormprods_trg = torch.cat(
                [mean_wnormprods_trg1, mean_wnormprods_trg2], dim=-1)
            assert mean_wnormprods_trg.shape == (n_seeds, n_srf, n_rsdls)
        else:
            assert n_rsdls == 1

            mean_wnormprods_trg = wnormprods_trg.mean(dim=-1, keepdim=True)
            assert mean_wnormprods_trg.shape == (n_seeds, n_srf, n_rsdls)

        # Linearly combining the reference and target predictions
        mean_wnormprods = (       w_trg  * mean_wnormprods_trg +
                           (1.0 - w_trg) * mean_wnormprods_mdl)
        assert mean_wnormprods.shape == (n_seeds, n_srf, n_rsdls)

        # Considering the surface areas
        pred_surfintegs = mean_wnormprods * areas.reshape(n_seeds, n_srf, 1)
        assert pred_surfintegs.shape == (n_seeds, n_srf, n_rsdls)

        # Getting the reference volume integrals
        ref_volintegs = problem.integrate_volumes(volsamps)
        assert ref_volintegs.shape == (n_seeds, n_srf)

        # Getting the residual terms
        resterms = pred_surfintegs - ref_volintegs.reshape(n_seeds, n_srf, 1)
        assert resterms.shape == (n_seeds, n_srf, n_rsdls)

        # Multiplying the residual terms
        if do_dblsampling:
            resterms_prod = resterms.prod(dim=-1)
            assert resterms_prod.shape == (n_seeds, n_srf)
        else:
            resterms_prod = torch.square(resterms).squeeze(-1)
            assert resterms_prod.shape == (n_seeds, n_srf)

        # Computing the main loss
        loss_main = resterms_prod.mean(-1)
        assert loss_main.shape == (n_seeds,)

        if do_bootstrap:
            with torch.no_grad():
                u_mdl_prime = target(points_mdl)
            loss_trgreg = torch.square(u_mdl - u_mdl_prime).mean([-3, -2, -1])
            assert loss_trgreg.shape == (n_seeds,)
        else:
            loss_trgreg = torch.zeros(n_seeds, device=tch_device, dtype=tch_dtype)
            assert loss_trgreg.shape == (n_seeds,)

        # The initial condition loss
        renew_ic = (epoch == 0) if (ic_frq == 0) else (epoch % ic_frq == 0)
        if ic_needsampling and renew_ic and (ic_dstr is not None):
            with torch.no_grad():
                if ic_dstr == 'sphere':
                    ic_normsamps =rng.normal((n_seeds, ic_n, dim))
                    assert ic_normsamps.shape == (n_seeds, ic_n, dim)

                    ic_ptstilde = ic_normsamps / ic_normsamps.norm()
                    assert ic_ptstilde.shape == (n_seeds, ic_n, dim)

                    assert ic_c.shape == (n_seeds, ic_n, dim)
                    assert ic_r.shape == (n_seeds, ic_n, 1)

                    ic_allpnts = ic_c + ic_ptstilde * ic_r
                    assert ic_allpnts.shape == (n_seeds, ic_n, dim)
                elif ic_dstr == 'trnvol':
                    icvols = volsampler(n=ic_n)
                    assert icvols['type'] == 'ball'

                    ic_c = icvols['centers']
                    assert ic_c.shape == (n_seeds, ic_n, dim)

                    ic_r_ = icvols['radii']
                    assert ic_r_.shape == (n_seeds, ic_n)

                    ic_r = ic_r_.unsqueeze(dim=-1)
                    assert ic_r.shape == (n_seeds, ic_n, 1)

                    untrd = rng.uniform((n_seeds, ic_n, 1))
                    assert untrd.shape == (n_seeds, ic_n, 1)

                    untr = untrd.pow(1.0 / dim)
                    assert untr.shape == (n_seeds, ic_n, 1)

                    ic_pntrs = untr * ic_r
                    assert ic_pntrs.shape == (n_seeds, ic_n, 1)

                    ic_theta = rng.normal((n_seeds, ic_n, dim))
                    assert ic_theta.shape == (n_seeds, ic_n, dim)

                    ic_thtilde = ic_theta / ic_theta.norm(dim=-1, keepdim=True)
                    assert ic_thtilde.shape == (n_seeds, ic_n, dim)

                    ic_allpnts = ic_c + ic_thtilde * ic_pntrs
                    assert ic_allpnts.shape == (n_seeds, ic_n, dim)
                else:
                    raise ValueError(f'ic/dstr={ic_dstr} not defined')

                ic_allgtvs = get_prob_sol(problem, ic_allpnts, eval_bs, 
                    get_field=False, out_lib='torch')['v']
                assert ic_allgtvs.shape == (n_seeds, ic_n)

        if ic_needsampling:
            ic_idxs = ((np.arange(ic_bs) + epoch * ic_bs) % ic_n).tolist()

            ic_pnts = ic_allpnts[:, ic_idxs, :]
            assert ic_pnts.shape == (n_seeds, ic_bs, dim)

            ic_vpreds = model(ic_pnts).squeeze(dim=-1)
            assert ic_vpreds.shape == (n_seeds, ic_bs)

            ic_gtvs = ic_allgtvs[:, ic_idxs]
            assert ic_gtvs.shape == (n_seeds, ic_bs)
        elif (ic_dstr is not None):
            ic_pnts = points_mdl.reshape(n_seeds, n_srf * n_srfpts_mdl, dim)
            assert ic_pnts.shape == (n_seeds, ic_bs, dim)

            ic_vpreds = u_mdl.reshape(n_seeds, n_srf * n_srfpts_mdl)
            assert ic_vpreds.shape == (n_seeds, ic_bs)

            with torch.no_grad():
                ic_gtvs = get_prob_sol(problem, ic_pnts, eval_bs, 
                    get_field=False, out_lib='torch')['v']
            assert ic_gtvs.shape == (n_seeds, ic_bs)

        if ic_bpp == 'bias':
            mdl_bias = model.layer_last[1].squeeze(dim=-1)
            assert mdl_bias.shape == (n_seeds, 1)

            ic_vpreds = ic_vpreds.detach() - mdl_bias.detach() + mdl_bias
            assert ic_vpreds.shape == (n_seeds, ic_bs)
        elif ic_bpp == 'all':
            pass
        else:
            raise RuntimeError(f'ic/bpp={ic_bpp} not defined')

        if ic_dstr is not None:
            loss_ic = torch.square(ic_vpreds - ic_gtvs).mean(dim=-1)
            assert loss_ic.shape == (n_seeds,)
        else:
            loss_ic = torch.zeros(n_seeds, dtype=tch_dtype, device=tch_device)
            assert loss_ic.shape == (n_seeds,)

        # The total loss
        loss = loss_main + w_trgreg * loss_trgreg + w_ic * loss_ic
        assert loss.shape == (n_seeds,)

        loss_sum = loss.sum()
        loss_sum.backward()

        # We will not update in the first epoch so that we will 
        # record the initialization statistics as well. Instead, 
        # we will update an extra epoch at the end.
        if (epoch > 0):
            opt.step()

        # Updating the target network
        if do_bootstrap and (epoch > 0):
            model_sd = model.state_dict()
            target_sd = target.state_dict()
            newtrg_sd = dict()
            with torch.no_grad():
                for key, param in model_sd.items():
                    param_trg = target_sd[key]
                    newtrg_sd[key] = tau * param_trg + (1-tau) * param
            target.load_state_dict(newtrg_sd)

        # computing the normal product variances
        with torch.no_grad(): 
            normprods = torch.cat([normprods_mdl, normprods_trg], dim=-1)
            npvm = (normprods.var(dim=-1)*areas.square()).mean(-1)

        # evaluating the performance of the model and target    
        perf_dict = dict()
        eval_strg = dict()
        for eid, eopts in evalprms.items():
            edstr = eopts['dstr']
            n_evlpnts = eopts['n']
            e_frq = eopts['frq']
            e_store = eopts['store']

            if (epoch % e_frq) > 0:
                assert eid in last_perfdict
                perf_dict[eid] = last_perfdict[eid]
                continue

            # Sampling the evaluation points
            with torch.no_grad():
                is_jointstatic = (eopts.get('rx/dstr', 'joint') == 'joint') and eopts.get('rx/static', False)
                if is_jointstatic and (epoch > 0):
                    e_pnts = eopts['e_pnts']
                    assert e_pnts.shape == (n_seeds, n_evlpnts, dim)
                elif edstr == 'uniform':
                    e_bias = eopts['bias']
                    assert e_bias.shape == (n_seeds, 1, dim)

                    e_slope = eopts['slope']
                    assert e_slope.shape == (n_seeds, 1, dim)

                    e_unfpnts = erng.uniform((n_seeds, n_evlpnts, dim))
                    assert e_unfpnts.shape == (n_seeds, n_evlpnts, dim)

                    e_pnts = e_bias + e_unfpnts * e_slope
                    assert e_pnts.shape == (n_seeds, n_evlpnts, dim)
                elif edstr in ('ball', 'trnvol'):
                    # Whether the point centers and radii are static
                    e_iscrstatic = eopts['iscrstatic']
                    if e_iscrstatic:
                        e_pntcs = eopts['pnt_c']
                        assert e_pntcs.shape == (n_seeds, n_evlpnts, dim)
                        
                        e_pntrs = eopts['pnt_r']
                        assert e_pntrs.shape == (n_seeds, n_evlpnts, 1)
                    elif (edstr == 'ball') and not(e_iscrstatic):
                        e_pntcs = eopts['c_xpnd']
                        assert e_pntcs.shape == (n_seeds, n_evlpnts, dim)

                        e_r = eopts['r_xpnd']
                        assert e_r.shape == (n_seeds, n_evlpnts, 1)
                        
                        euntrd = erng.uniform((n_seeds, n_evlpnts, 1))
                        assert euntrd.shape == (n_seeds, n_evlpnts, 1)
                    
                        untr = euntrd.pow(1.0 / dim)
                        assert untr.shape == (n_seeds, n_evlpnts, 1)

                        e_pntrs = untr * e_r
                        assert e_pntrs.shape == (n_seeds, n_evlpnts, 1)
                    elif (edstr == 'trnvol') and not(e_iscrstatic):
                        evols = volsampler(n=n_evlpnts)
                        assert evols['type'] == 'ball'

                        e_pntcs = evols['centers']
                        assert e_pntcs.shape == (n_seeds, n_evlpnts, dim)

                        e_r_ = evols['radii']
                        assert e_r_.shape == (n_seeds, n_evlpnts)

                        e_r = e_r_.unsqueeze(dim=-1)
                        assert e_r.shape == (n_seeds, n_evlpnts, 1)
                        
                        euntrd = erng.uniform((n_seeds, n_evlpnts, 1))
                        assert euntrd.shape == (n_seeds, n_evlpnts, 1)
                    
                        untr = euntrd.pow(1.0 / dim)
                        assert untr.shape == (n_seeds, n_evlpnts, 1)

                        e_pntrs = untr * e_r
                        assert e_pntrs.shape == (n_seeds, n_evlpnts, 1)
                    else:
                        raise RuntimeError(f'case not defined')                    
                    
                    # Whether the point theta-tildes are static (i.e., determined 
                    # only once at the beginning of training)
                    e_isthstatic = eopts['isthstatic']
                    if e_isthstatic:
                        e_thtilde = eopts['thtilde']
                        assert e_thtilde.shape == (n_seeds, n_evlpnts, dim)
                    else:
                        etheta = erng.normal((n_seeds, n_evlpnts, dim))
                        assert etheta.shape == (n_seeds, n_evlpnts, dim)

                        e_thtilde = etheta / etheta.norm(dim=-1, keepdim=True)
                        assert e_thtilde.shape == (n_seeds, n_evlpnts, dim)

                    e_pnts = e_pntcs + e_thtilde * e_pntrs
                    assert e_pnts.shape == (n_seeds, n_evlpnts, dim)
                elif edstr in ('grid'):
                    e_pnts = eopts['pnts']
                    assert e_pnts.shape == (n_seeds, n_g, dim)
                else:
                    raise RuntimeError(f'eval dstr "{edstr}" not implmntd')
                if is_jointstatic and (epoch == 0):
                    eopts['e_pnts'] = e_pnts.detach().clone() 

            # Computing the model, target and ground truth solutions
            prob_sol = get_prob_sol(problem, e_pnts, n_eval=eval_bs, 
                get_field=False, out_lib='torch')

            e_prbsol = prob_sol['v']
            assert e_prbsol.shape == (n_seeds, n_evlpnts)

            # Computing the model solution
            with torch.no_grad():
                mdl_sol = get_nn_sol(model, e_pnts, n_eval=eval_bs,
                    get_field=False, out_lib='torch')

            e_mdlsol = mdl_sol['v']
            assert e_mdlsol.shape == (n_seeds, n_evlpnts)
            
            # Computing the target solution
            if do_bootstrap:
                with torch.no_grad():
                    trg_sol = get_nn_sol(target, e_pnts, n_eval=eval_bs, 
                        get_field=False, out_lib='torch')

                e_trgsol = trg_sol['v']
                assert e_trgsol.shape == (n_seeds, n_evlpnts)

            eperfs = dict()
            eperfs['mdl'] = get_perfdict(e_pnts, e_mdlsol, e_prbsol)
            if do_bootstrap:
                eperfs['trg'] = get_perfdict(e_pnts, e_trgsol, e_prbsol)
            eperfs = deep2hie(eperfs, dictcls=dict)
            # Example: eperfs = {'mdl/pln/mse': ...,
            #                    'mdl/pln/mae': ...,
            #                    'mdl/bc/mse': ...,
            #                    'mdl/bc/mae': ...,
            #                    'mdl/slc/mse': ...,
            #                    'mdl/slc/mae': ...,
            #                    'trg/pln/mse': ...,
            #                    'trg/pln/mae': ...,
            #                    'trg/bc/mse': ...,
            #                    'trg/bc/mae': ...,
            #                    'trg/slc/mse': ...,
            #                    'trg/slc/mae': ...,
            #                   }
            perf_dict[eid] = eperfs
            last_perfdict[eid] = eperfs
            
            if do_logtb:
                for kk, vv in eperfs.items():
                    tbwriter.add_scalar(f'perf/{eid}/{kk}', vv.mean(), epoch)
            
            # Storing the evaluation results
            if e_store:
                e_strg = dict()
                e_strg['sol/mdl'] = e_mdlsol
                if do_bootstrap:
                    e_strg['sol/trg'] = e_trgsol
                e_strg['sol/gt'] = e_prbsol
                if edstr != 'grid':
                    e_strg['pnts'] = e_pnts
                e_strg = {kk: vv.detach().cpu().numpy().astype(np.float16) 
                          for kk, vv in e_strg.items()}
                eval_strg[eid] = e_strg

                if do_logtb and (edstr == 'grid') and (dim == 2):
                    soltd_list = [('mdl', mdl_sol, fig_mdl, ax_mdl, cax_mdl)]
                    if do_bootstrap:
                        soltd_list += [('trg', trg_sol, fig_trg, ax_trg, cax_trg)]
                    soltd_list += [('gt', prob_sol, fig_gt, ax_gt, cax_gt)]
                    for sol_t, sol_dict, fig, ax, cax in soltd_list:
                        x1_msh_np, x2_msh_np = eopts['xi_msh_np']
                        plot_sol(x1_msh_np, x2_msh_np, sol_dict, fig=fig, ax=ax, cax=cax)
                        fig.set_tight_layout(True)
                        tbwriter.add_figure(f'viz/{eid}/{sol_t}', fig, epoch)
                    tbwriter.flush()
         
        # monitoring the resource utilization 
        if epoch % iomon_period == 0:
            s_rsrc = resource.getrusage(resource.RUSAGE_SELF)
            c_rsrc = resource.getrusage(resource.RUSAGE_CHILDREN)
            
            psmem = psutil.virtual_memory()
            pscpu = psutil.cpu_times()
            pscpuload = psutil.getloadavg()
            mon_dict = {'cpu/mem/tot': [psmem.total] * n_seeds, 
                'cpu/mem/avail': [psmem.available] * n_seeds, 
                'cpu/mem/used': [psmem.used] * n_seeds,
                'cpu/mem/free': [psmem.free] * n_seeds,
                'cpu/time/user/ps': [pscpu.user] * n_seeds,
                'cpu/time/sys/ps': [pscpu.system] * n_seeds,
                'cpu/time/idle/ps': [pscpu.idle] * n_seeds,
                'cpu/load/1m': [pscpuload[0]] * n_seeds,
                'cpu/load/5m': [pscpuload[1]] * n_seeds,
                'cpu/load/15m': [pscpuload[2]] * n_seeds,
                'cpu/time/train': [time.time()   - trn_sttime] * n_seeds,
                'cpu/time/sys/py':   [s_rsrc.ru_stime  + c_rsrc.ru_stime] * n_seeds,
                'cpu/time/user/py':  [s_rsrc.ru_utime  + c_rsrc.ru_utime] * n_seeds,
                'n_seeds': [n_seeds] * n_seeds}
            if 'cuda' in device_name:
                t_gpumem = torch.cuda.get_device_properties(tch_device).total_memory
                r_gpumem = torch.cuda.memory_reserved(tch_device)
                a_gpumem = torch.cuda.memory_allocated(tch_device)
                f_gpumem = r_gpumem - a_gpumem
                mon_dict.update({'gpu/mem/tot':   [t_gpumem] * n_seeds,
                                 'gpu/mem/res':   [r_gpumem] * n_seeds,
                                 'gpu/mem/alloc': [a_gpumem] * n_seeds,
                                 'gpu/mem/free':  [f_gpumem] * n_seeds})

        # pushing the results to the data writer
        psld = deep2hie({'perf': perf_dict}, odict)
        slst = [('loss/total',  loss.tolist()),
                ('loss/main',   loss_main.tolist()),
                ('loss/trgreg', loss_trgreg.tolist()),
                ('loss/ic',     loss_ic.tolist()),
                ('npvm',        npvm.tolist()),
                *list(psld.items())]
        stat_dict = odict(slst)
        for stat_name, stat_vals in stat_dict.items():
            avg_history.setdefault(stat_name, [])
            avg_history[stat_name].append(stat_vals)

        dtups = []
        if epoch % io_avgfrq == 0:
            avg_statlst  = [('epoch',       [epoch] * n_seeds),
                            ('rng_seed',    rng_seeds.tolist())]
            avg_statlst += [(name, np.stack(svl, axis=0).mean(axis=0).tolist())
                             for name, svl in avg_history.items()]
            avg_statdict = odict(avg_statlst)

            dtups += [('hp',    hp_dict,      'pd.cat'),
                      ('stat',  avg_statdict, 'pd.qnt'),
                      ('etc',   etc_dict,     'pd.cat')]
            avg_history = odict()
            
        for eid, e_strg in eval_strg.items():
            msg_ =  f'eval/{eid} requires storage, thus "eval/{eid}/frq" '
            msg_ += f'% "io/avg/frq" == 0 should hold.'
            assert epoch % io_avgfrq == 0, msg_
            dtups += [(f'var/eval/{eid}', e_strg, 'np.arr')]
        
        if epoch % iomon_period == 0:
            assert epoch % io_avgfrq == 0
            dtups += [('mon', mon_dict, 'pd.qnt')]

        if epoch % chkpnt_period == 0:
            assert epoch % io_avgfrq == 0
            mdl_sdnp = {k: v.detach().cpu().numpy() 
                for k, v in model.state_dict().items()}
            dtups += [('mdl',   mdl_sdnp, 'np.arr')]
            if do_bootstrap:
                trg_sdnp = {k: v.detach().cpu().numpy() 
                    for k, v in target.state_dict().items()}
                dtups += [('trg',   trg_sdnp, 'np.arr')]

        dwriter.add(data_tups=dtups, file_path=hdfpth)

        # Computing the loss moving averages
        loss_ema_mean, loss_ema_std_mean = ema('loss', loss)
        npvm_ema_mean, npvm_ema_std_mean = ema('npvm', npvm)
        if (epoch % 1000 == 0) and (results_dir is not None):
            print_str = f'Epoch {epoch}, EMA loss = {loss_ema_mean:.4f}'
            print_str += f' +/- {2*loss_ema_std_mean:.4f}'
            print_str += f', EMA Field-Norm Product Variance = {npvm_ema_mean:.4f}'
            print_str += f' +/- {2*npvm_ema_std_mean:.4f} ({time.time()-trn_sttime:0.1f} s)'
            print(print_str, flush=True)

        if do_logtb:
            import logging
            logging.getLogger("tensorboardX.x2num").setLevel(logging.CRITICAL)  
            tbwriter.add_scalar('loss/total', loss.mean(), epoch)
            tbwriter.add_scalar('loss/main', loss_main.mean(), epoch)
            tbwriter.add_scalar('loss/trgreg', loss_trgreg.mean(), epoch)
            tbwriter.add_scalar('loss/ic', loss_ic.mean(), epoch)
            tbwriter.add_scalar('loss/npvm', npvm.mean(), epoch)

        if do_tchsave and (epoch % chkpnt_period == 0):
            model_history[epoch] = deepcopy({k: v.cpu() for k, v
                in model.state_dict().items()})
            target_history[epoch] = deepcopy({k: v.cpu() for k, v
                in target.state_dict().items()})
            

    if results_dir is not None:
        print(f'Training finished in {time.time() - trn_sttime:.1f} seconds.')
    dwriter.close()
    if do_logtb:
        tbwriter.flush()
    
    outdict = dict()
    tchmemusage = profmem()
    assert str(tch_device) in tchmemusage
    if 'cuda' in device_name:
        tch_dvcmem = torch.cuda.get_device_properties(tch_device).total_memory
    else:
        tch_dvcmem = os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES')
    outdict['dvc/mem/alloc'] = tchmemusage[str(tch_device)]
    outdict['dvc/mem/total'] = tch_dvcmem

In [ ]:
    fig = None
    has_grid = any(eopts.get('dstr', '') == 'grid' 
                   for eid, eopts in evalprms.items())
    if (storage_dir is not None) and has_grid:
        soltd_list = [('mdl', mdl_sol, fig_mdl, ax_mdl, cax_mdl)]
        eopts = list(eopts for eid, eopts in evalprms.items()
                     if eopts.get('dstr', '') == 'grid')[0]
        e_pnts = eopts['pnts']
        x1_msh_np, x2_msh_np = eopts['xi_msh_np']

        n_rows, n_cols = 1, 2 + do_bootstrap
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(
            n_cols * 3.5, n_rows * 3), dpi=72)
        cax = None

        # Computing the model, target and ground truth solutions
        prob_sol = get_prob_sol(problem, e_pnts, n_eval=eval_bs, get_field=False)
        with torch.no_grad():
            mdl_sol = get_nn_sol(model, e_pnts, n_eval=eval_bs, get_field=False) 
            if do_bootstrap:
                trg_sol = get_nn_sol(target, e_pnts, n_eval=eval_bs, get_field=False)

        soltd_list = [('gt', prob_sol, axes[0], 'Ground Truth'),
                      ('mdl', mdl_sol, axes[1], 'Prediction')]
        if do_bootstrap:
            soltd_list += [('trg', trg_sol, axes[2], 'Target')]
        for sol_t, sol_dict, ax, ttl in soltd_list:
            plot_sol(x1_msh_np, x2_msh_np, sol_dict, fig=fig, ax=ax, cax=cax)
            ax.set_title(ttl)
    fig

In [ ]:
    if do_tchsave:
        torch.save(model_history, f'{cfgstrg_dir}/ckpt_mdl.pt')
        if do_bootstrap:
            torch.save(target_history, f'{cfgstrg_dir}/ckpt_trg.pt')
    if storage_dir is not None:
        shutil.copy2(hdfpth, f'{cfgstrg_dir}/progress.h5')
        if fig is not None:
            fig.savefig(f'{cfgstrg_dir}/finalpred.pdf', dpi=144, bbox_inches="tight")   
    if do_profile:
        profiler.stop()
        html = profiler.output_html()
        htmlpath = f'{cfgstrg_dir}/profiler.html'
        with open(htmlpath, 'w') as fp:
            fp.write(html.encode('ascii', errors='ignore').decode('ascii'))